In [ ]:
# Блок 1: Задача Классификации Изображений (PyTorch)

# Что такое Классификация Изображений?
# Это задача Computer Vision по присвоению изображению одной метки (класса)
# из предопределенного набора (например, "кошка", "собака").
# Вход: Изображение (представленное как тензор в PyTorch).
# Выход: Индекс предсказанного класса или вероятности для каждого класса.

# Цель Классификации:
# Автоматически категоризировать изображения.

# --------------------------------------------------

# Блок 2: Основные Подходы (Фокус на PyTorch)

# Современный подход (Deep Learning с PyTorch):
# - Сверточные Нейронные Сети (Convolutional Neural Networks - CNNs)
# - PyTorch предоставляет мощные инструменты для построения, обучения и развертывания CNN:
#   - `torch.nn`: Модули для слоев (Conv2d, Linear, ReLU, MaxPool2d и т.д.).
#   - `torchvision`: Утилиты для CV (датасеты, трансформеры, предобученные модели).
#   - `torch.optim`: Оптимизаторы (Adam, SGD).
#   - `torch.utils.data.DataLoader`: Эффективная загрузка данных.

# --------------------------------------------------

# Блок 3: CNNs для Классификации в PyTorch

# Ключевые слои (`torch.nn`):

# 1. Сверточный слой (Convolutional Layer):
#    - Применяет фильтры для извлечения пространственных признаков.
#    - `in_channels`: Количество каналов входного изображения/карты признаков.
#    - `out_channels`: Количество фильтров (определяет глубину выходной карты признаков).
#    - `kernel_size`: Размер фильтра (например, 3 или (3, 3)).
#    - `stride`: Шаг перемещения фильтра.
#    - `padding`: Добавление пикселей по краям для контроля размера выхода.
import torch
import torch.nn as nn

conv_layer = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)

# 2. Функция Активации (Activation Function):
#    - Вносит нелинейность.
#    - Применяются как модули `torch.nn` или функции из `torch.nn.functional`.
relu_activation = nn.ReLU()
# или
# import torch.nn.functional as F
# output = F.relu(input_tensor)

# 3. Слой Субдискретизации (Pooling Layer):
#    - Уменьшает пространственный размер.
#    - `kernel_size`: Размер окна пулинга.
#    - `stride`: Шаг окна пулинга.
pool_layer = nn.MaxPool2d(kernel_size=2, stride=2)

# 4. Слой Выравнивания (Flatten):
#    - Преобразует многомерный тензор в 1D вектор для полносвязных слоев.
#    - Можно использовать `nn.Flatten()` или `tensor.view(batch_size, -1)`.
flatten_layer = nn.Flatten()
# или
# flattened_tensor = output_from_pooling.view(output_from_pooling.size(0), -1)

# 5. Полносвязный слой (Fully Connected / Dense Layer):
#    - Применяет линейное преобразование к входному вектору.
#    - `in_features`: Размер входного вектора (выход flatten).
#    - `out_features`: Количество нейронов в слое.
dense_layer = nn.Linear(in_features=128 * 7 * 7, out_features=512) # Пример размеров

# 6. Выходной слой (Output Layer):
#    - Полносвязный слой с количеством нейронов = количество классов.
#    - Для мультиклассовой классификации: активация Softmax неявно встроена в `nn.CrossEntropyLoss`, поэтому последний слой обычно просто `nn.Linear`.
#    - Для бинарной классификации: `nn.Linear(..., out_features=1)` + `nn.Sigmoid()` (или использовать `nn.BCEWithLogitsLoss` без Sigmoid).
output_layer_multi = nn.Linear(in_features=512, out_features=10) # 10 классов
output_layer_binary = nn.Linear(in_features=512, out_features=1)
sigmoid_activation = nn.Sigmoid()

# Определение Модели в PyTorch:
# - Наследуемся от `nn.Module`.
# - Определяем слои как атрибуты в `__init__`.
# - Определяем прямой проход (forward pass) в методе `forward`.

class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        # Определяем слои здесь
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2, 2)
        self.flatten = nn.Flatten()
        # Размер после пулинга нужно рассчитать или определить динамически
        # Например, для входа 32x32: 32 -> pool(16) -> pool(8). Размер = 32 * 8 * 8
        # self.fc1 = nn.Linear(32 * 8 * 8, 128)
        # self.relu3 = nn.ReLU()
        # self.fc2 = nn.Linear(128, num_classes)
        # Вместо жесткого кодирования размера можно использовать nn.AdaptiveAvgPool2d
        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, 1)) # Сводит к 1x1
        self.fc = nn.Linear(32 * 1 * 1, num_classes) # Теперь размер известен

    def forward(self, x):
        # Определяем последовательность применения слоев
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        # x = self.flatten(x)
        # x = self.relu3(self.fc1(x))
        # x = self.fc2(x)
        x = self.adaptive_pool(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

# --------------------------------------------------

# Блок 4: Процесс Обучения и Использования (PyTorch)

# 1. Сбор и Подготовка Данных (`torchvision`):
#    - `torchvision.datasets`: Загрузка стандартных датасетов (CIFAR10, ImageNet) или своих (`ImageFolder`).
#    - `torchvision.transforms`: Предобработка и аугментация изображений.
#      - `transforms.ToTensor()`: Конвертирует PIL Image/numpy array в PyTorch тензор и масштабирует в [0, 1].
#      - `transforms.Normalize(mean, std)`: Нормализует тензор.
#      - `transforms.Resize()`, `transforms.CenterCrop()`, `transforms.RandomHorizontalFlip()`, etc.
#    - `torch.utils.data.Dataset`: Базовый класс для создания своих датасетов.
#    - `torch.utils.data.DataLoader`: Загружает данные батчами, перемешивает, использует несколько воркеров.
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Пример трансформаций
train_transform = transforms.Compose([
    transforms.Resize((224, 224)), # Пример размера для ResNet
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Стандартные для ImageNet
])
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Пример загрузки с ImageFolder (ожидает структуру: root/class_a/img1.jpg, root/class_b/img2.jpg)
# train_dataset = datasets.ImageFolder(root='path/to/train', transform=train_transform)
# val_dataset = datasets.ImageFolder(root='path/to/validation', transform=val_transform)

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
# val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

# 2. Определение/Выбор Модели:
#    - Создать свою (`nn.Module`).
#    - Использовать предобученную из `torchvision.models`.
import torchvision.models as models

# Пример использования предобученного ResNet18
# model = models.resnet18(pretrained=True)
# # Заменить последний слой для своего количества классов
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, num_classes) # num_classes - ваше количество классов

# 3. Определение Устройств, Функции Потерь и Оптимизатора:
#    - Перенос модели и данных на GPU (если доступно).
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

#    - Функция потерь (`torch.nn`).
criterion = nn.CrossEntropyLoss() # Для мультиклассовой классификации
# criterion = nn.BCEWithLogitsLoss() # Для бинарной (более стабильна, чем Sigmoid + BCELoss)

#    - Оптимизатор (`torch.optim`).
import torch.optim as optim
# optimizer = optim.Adam(model.parameters(), lr=0.001)
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# 4. Цикл Обучения (Training Loop):
#    - Итерация по эпохам.
#    - Внутри эпохи итерация по батчам из `DataLoader`.
#    - Перенос данных на `device`.
#    - Обнуление градиентов (`optimizer.zero_grad()`).
#    - Прямой проход (`outputs = model(inputs)`).
#    - Вычисление потерь (`loss = criterion(outputs, labels)`).
#    - Обратный проход (`loss.backward()`).
#    - Шаг оптимизатора (`optimizer.step()`).
#    - (Опционально) Расчет метрик (accuracy).

# # --- Псевдокод Тренировочного Цикла ---
# num_epochs = 10
# for epoch in range(num_epochs):
#     model.train() # Перевести модель в режим обучения
#     running_loss = 0.0
#     correct_train = 0
#     total_train = 0
#     for i, data in enumerate(train_loader, 0):
#         inputs, labels = data[0].to(device), data[1].to(device)
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
#
#         running_loss += loss.item()
#         # Расчет accuracy (пример)
#         _, predicted = torch.max(outputs.data, 1) # Для CrossEntropyLoss
#         # Для BCEWithLogitsLoss: predicted = (torch.sigmoid(outputs.data) > 0.5).float()
#         total_train += labels.size(0)
#         correct_train += (predicted == labels).sum().item()
#
#     epoch_loss = running_loss / len(train_loader)
#     epoch_acc = 100 * correct_train / total_train
#     print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.2f}%')
#
#     # --- Валидационный Цикл ---
#     model.eval() # Перевести модель в режим оценки (отключает dropout, batchnorm использует накопленную статистику)
#     val_loss = 0.0
#     correct_val = 0
#     total_val = 0
#     with torch.no_grad(): # Отключить вычисление градиентов
#         for data in val_loader:
#             inputs, labels = data[0].to(device), data[1].to(device)
#             outputs = model(inputs)
#             loss = criterion(outputs, labels)
#             val_loss += loss.item()
#             _, predicted = torch.max(outputs.data, 1)
#             total_val += labels.size(0)
#             correct_val += (predicted == labels).sum().item()
#
#     val_epoch_loss = val_loss / len(val_loader)
#     val_epoch_acc = 100 * correct_val / total_val
#     print(f'Epoch {epoch+1}/{num_epochs}, Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.2f}%')
# # --- Конец Псевдокода ---

# 5. Оценка Модели (Evaluation):
#    - Аналогично валидационному циклу, но на тестовой выборке.
#    - Использование `model.eval()` и `torch.no_grad()`.
#    - Расчет финальных метрик (Accuracy, Precision, Recall, F1, Confusion Matrix).

# 6. Использование Модели (Inference/Prediction):
#    - Загрузка обученных весов (`model.load_state_dict(torch.load(PATH))`).
#    - Перевод модели в режим оценки (`model.eval()`).
#    - Подготовка входного изображения (трансформации, добавление batch dimension, перенос на `device`).
#    - Отключение градиентов (`with torch.no_grad():`).
#    - Получение выхода модели (`outputs = model(input_tensor)`).
#    - Интерпретация выхода:
#      - `torch.max(outputs, 1)` -> получение индекса класса с максимальной логитом/вероятностью.
#      - `torch.softmax(outputs, dim=1)` -> получение вероятностей для всех классов.
#      - `torch.sigmoid(outputs)` -> получение вероятности для бинарной классификации (если использовался `nn.Linear` без Sigmoid на выходе).

# --------------------------------------------------

# Блок 5: Популярные Датасеты (см. предыдущий ответ)
# - MNIST, Fashion-MNIST, CIFAR-10, CIFAR-100, ImageNet, etc.
# - `torchvision.datasets` имеет встроенные классы для многих из них.

# --------------------------------------------------

# Блок 6: Метрики Оценки (см. предыдущий ответ)
# - Accuracy, Precision, Recall, F1-Score, Confusion Matrix, AUC-ROC.
# - Библиотеки вроде `scikit-learn` могут помочь в расчете этих метрик из выходов модели и истинных меток.

# --------------------------------------------------

# Блок 7: Пример Задачи и Решения (Концептуально, PyTorch)

# --- Условие Задачи ---
# Задача: Классифицировать изображения из датасета CIFAR-10 (10 классов: самолет, автомобиль, птица, кошка, олень, собака, лягушка, лошадь, корабль, грузовик).
# Использовать PyTorch и `torchvision`.

# --- Решение (концептуальное, PyTorch) ---

# Шаг 1: Импорт библиотек
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Шаг 2: Подготовка данных (CIFAR-10)
# Трансформации (примерные)
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) # Нормализация для CIFAR

batch_size = 64

# Загрузка датасетов
# trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
#                                         download=True, transform=transform)
# trainloader = DataLoader(trainset, batch_size=batch_size,
#                                           shuffle=True, num_workers=2)
#
# testset = torchvision.datasets.CIFAR10(root='./data', train=False,
#                                        download=True, transform=transform)
# testloader = DataLoader(testset, batch_size=batch_size,
#                                          shuffle=False, num_workers=2)
#
# classes = ('plane', 'car', 'bird', 'cat', 'deer',
#            'dog', 'frog', 'horse', 'ship', 'truck')
num_classes = 10

# Шаг 3: Определение архитектуры CNN модели
# Можно использовать SimpleCNN из Блока 3 или предобученную
# model = SimpleCNN(num_classes=num_classes)
# Или, например, ResNet18
# model = models.resnet18(pretrained=False) # Обучаем с нуля или pretrained=True для transfer learning
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, num_classes)

# Шаг 4: Определение устройства, потерь и оптимизатора
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model.to(device)
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Шаг 5: Обучение модели
# # Используем цикл обучения, как в псевдокоде из Блока 4
# # ... (код цикла обучения с trainloader и testloader для валидации) ...
# print('Finished Training')
#
# # (Опционально) Сохранение модели
# # PATH = './cifar_net.pth'
# # torch.save(model.state_dict(), PATH)

# Шаг 6: Оценка модели на тестовой выборке
# correct = 0
# total = 0
# model.eval() # Перевод в режим оценки
# with torch.no_grad():
#     for data in testloader:
#         images, labels = data[0].to(device), data[1].to(device)
#         outputs = model(images)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()
#
# accuracy = 100 * correct / total
# print(f'Accuracy of the network on the 10000 test images: {accuracy:.2f} %')

# Шаг 7: Использование модели для предсказания на одном изображении
# # Загрузка модели (если нужно)
# # model = SimpleCNN(num_classes=num_classes) # Создать экземпляр той же архитектуры
# # model.load_state_dict(torch.load(PATH))
# # model.to(device)
# model.eval()
#
# # Получение одного изображения из тестового набора
# dataiter = iter(testloader)
# images, labels = next(dataiter)
# image_to_predict = images[0].unsqueeze(0).to(device) # Взять первое, добавить batch dim, отправить на device
# true_label = labels[0]
#
# # Предсказание
# with torch.no_grad():
#     outputs = model(image_to_predict)
#     probabilities = torch.softmax(outputs, dim=1)
#     confidence, predicted_index = torch.max(probabilities, 1)
#
# predicted_class = classes[predicted_index.item()]
# confidence_percent = confidence.item() * 100
# true_class = classes[true_label.item()]
#
# print(f'Predicted: "{predicted_class}" with {confidence_percent:.2f}% confidence.')
# print(f'True label: "{true_class}"')

# --- Конец Примера ---

# --------------------------------------------------

In [ ]:
# Блок 1: Введение в Архитектуры Классификации в torchvision

# `torchvision.models` предоставляет доступ к множеству популярных
# архитектур компьютерного зрения, включая предобученные на ImageNet модели.
# Использование предобученных моделей (Transfer Learning) - это очень
# эффективный подход, особенно когда ваш собственный датасет не очень большой.

# Основные идеи Transfer Learning:
# 1. Feature Extraction (Извлечение признаков):
#    - Используем предобученную модель как фиксированный извлекатель признаков.
#    - Замораживаем веса всех слоев, кроме последнего (классификатора).
#    - Обучаем только новый классификатор на своих данных.
#    - Подходит, когда ваш датасет мал или очень похож на ImageNet.
# 2. Fine-tuning (Дообучение):
#    - Инициализируем модель предобученными весами.
#    - Заменяем классификатор на новый.
#    - Размораживаем часть или все слои "тела" модели (backbone).
#    - Обучаем всю модель (или ее часть) на своих данных, обычно с маленькой скоростью обучения (learning rate).
#    - Подходит, когда ваш датасет больше или отличается от ImageNet.

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
# Предполагается, что у вас есть DataLoader'ы: train_loader, val_loader
# и определено количество классов: num_classes
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --------------------------------------------------

# Блок 2: AlexNet

# Краткое описание:
# Одна из первых глубоких сетей, выигравшая ImageNet LSVRC 2012.
# Показала эффективность CNN для сложных задач CV.
# Состоит из 5 сверточных и 3 полносвязных слоев. Использует ReLU, MaxPool, Dropout.
# Сейчас используется реже, но имеет историческое значение.

# Загрузка модели:
# Загрузить предобученную на ImageNet
alexnet_pretrained = models.alexnet(pretrained=True)
# Загрузить только архитектуру (случайные веса)
alexnet_scratch = models.alexnet(pretrained=False)

# Модификация для своих данных (Transfer Learning):
# AlexNet имеет блок 'classifier', последний слой - 6-й (индекс).
num_classes = 10 # Пример: для CIFAR-10
num_ftrs_alexnet = alexnet_pretrained.classifier[6].in_features
# Заменяем последний слой
alexnet_pretrained.classifier[6] = nn.Linear(num_ftrs_alexnet, num_classes)

# То же самое для модели с нуля
num_ftrs_scratch_alexnet = alexnet_scratch.classifier[6].in_features
alexnet_scratch.classifier[6] = nn.Linear(num_ftrs_scratch_alexnet, num_classes)

# Пример дообучения (Fine-tuning Setup - Концептуально):
# model_to_train = alexnet_pretrained.to(device)
# criterion = nn.CrossEntropyLoss()
# # Оптимизируем только параметры нового классификатора (Feature Extraction)
# # optimizer = optim.SGD(model_to_train.classifier[6].parameters(), lr=0.001, momentum=0.9)
# # Или оптимизируем все параметры (Full Fine-tuning, с малым lr)
# optimizer = optim.SGD(model_to_train.parameters(), lr=1e-4, momentum=0.9)
# # Далее следует стандартный цикл обучения PyTorch...
# # for epoch in range(num_epochs):
# #     model_to_train.train()
# #     for inputs, labels in train_loader:
# #         inputs, labels = inputs.to(device), labels.to(device)
# #         optimizer.zero_grad()
# #         outputs = model_to_train(inputs)
# #         loss = criterion(outputs, labels)
# #         loss.backward()
# #         optimizer.step()
# #     # Валидация с model_to_train.eval() и torch.no_grad()...

# --------------------------------------------------

# Блок 3: VGG (Visual Geometry Group)

# Краткое описание:
# Углубили AlexNet, используя очень маленькие сверточные фильтры (3x3).
# Показали, что глубина сети важна для производительности.
# Простая и однородная архитектура (стеки сверток 3x3 + MaxPool).
# Популярные варианты: VGG16, VGG19 (число означает количество весовых слоев).
# Достаточно "тяжелая" по количеству параметров.

# Загрузка модели:
# Загрузить предобученную VGG16
vgg16_pretrained = models.vgg16(pretrained=True)
# Загрузить только архитектуру VGG16
vgg16_scratch = models.vgg16(pretrained=False)

# Модификация для своих данных:
# VGG также имеет блок 'classifier', последний слой - 6-й.
num_classes = 100 # Пример: для CIFAR-100
num_ftrs_vgg = vgg16_pretrained.classifier[6].in_features
vgg16_pretrained.classifier[6] = nn.Linear(num_ftrs_vgg, num_classes)

num_ftrs_scratch_vgg = vgg16_scratch.classifier[6].in_features
vgg16_scratch.classifier[6] = nn.Linear(num_ftrs_scratch_vgg, num_classes)

# Пример дообучения (Fine-tuning Setup - Концептуально):
# model_to_train = vgg16_pretrained.to(device)
# criterion = nn.CrossEntropyLoss()
# # Оптимизация только классификатора
# # optimizer = optim.Adam(model_to_train.classifier[6].parameters(), lr=0.001)
# # Оптимизация всей сети (с малым lr)
# optimizer = optim.Adam(model_to_train.parameters(), lr=1e-5)
# # Далее стандартный цикл обучения...

# --------------------------------------------------

# Блок 4: ResNet (Residual Network)

# Краткое описание:
# Революционная архитектура, позволившая эффективно обучать очень глубокие сети (до 152 слоев и больше).
# Ввели "остаточные блоки" (residual blocks) с skip connections (проброс входа на выход блока).
# Это решает проблему затухания градиента в глубоких сетях.
# Очень популярный и сильный выбор для многих задач CV.
# Варианты: ResNet18, ResNet34, ResNet50, ResNet101, ResNet152.

# Загрузка модели:
# Загрузить предобученную ResNet50
resnet50_pretrained = models.resnet50(pretrained=True)
# Загрузить только архитектуру ResNet50
resnet50_scratch = models.resnet50(pretrained=False)

# Модификация для своих данных:
# У ResNet последний слой называется 'fc' (fully connected).
num_classes = 200 # Пример: для CUB-200 Birds
num_ftrs_resnet = resnet50_pretrained.fc.in_features
resnet50_pretrained.fc = nn.Linear(num_ftrs_resnet, num_classes)

num_ftrs_scratch_resnet = resnet50_scratch.fc.in_features
resnet50_scratch.fc = nn.Linear(num_ftrs_scratch_resnet, num_classes)

# Пример дообучения (Fine-tuning Setup - Концептуально):
# model_to_train = resnet50_pretrained.to(device)
# criterion = nn.CrossEntropyLoss()
#
# # --- Стратегия 1: Feature Extraction ---
# # Заморозить все слои, кроме последнего
# # for param in model_to_train.parameters():
# #     param.requires_grad = False
# # # Убедиться, что параметры нового слоя разморожены (они такие по умолчанию)
# # model_to_train.fc.requires_grad = True
# # # Оптимизировать только параметры fc слоя
# # optimizer = optim.Adam(model_to_train.fc.parameters(), lr=0.001)
#
# # --- Стратегия 2: Full Fine-tuning ---
# # Разморозить все слои (они разморожены по умолчанию после замены fc)
# # Оптимизировать все параметры, но с очень маленьким lr для backbone
# optimizer = optim.SGD(model_to_train.parameters(), lr=1e-4, momentum=0.9)
# # Можно задать разные lr для backbone и head (см. документацию optim)
#
# # Далее стандартный цикл обучения...

# --------------------------------------------------

# Блок 5: GoogLeNet / Inception

# Краткое описание:
# Ввели "Inception module", который выполняет свертки с разными размерами ядер (1x1, 3x3, 5x5) и MaxPool параллельно,
# а затем конкатенирует результаты. Это позволяет сети выбирать наиболее подходящие признаки на разных масштабах.
# Более эффективна по вычислениям, чем VGG.
# Использовала вспомогательные классификаторы во время обучения (в `torchvision` они активны только при обучении).
# `torchvision` предоставляет `googlenet` (Inception v1) и `inception_v3`.

# Загрузка модели:
# Загрузить предобученную GoogLeNet
googlenet_pretrained = models.googlenet(pretrained=True)
# Загрузить только архитектуру GoogLeNet
googlenet_scratch = models.googlenet(pretrained=False)

# Модификация для своих данных:
# У GoogLeNet последний слой также называется 'fc'.
num_classes = 50 # Пример
num_ftrs_googlenet = googlenet_pretrained.fc.in_features
googlenet_pretrained.fc = nn.Linear(num_ftrs_googlenet, num_classes)
# Важно: Если используете pretrained=True, вспомогательные классификаторы (aux1, aux2)
# тоже нужно модифицировать или отключить, если они есть в используемой версии.
# В стандартной реализации torchvision они обычно не мешают при inference (model.eval()).
# При обучении с нуля (pretrained=False) их тоже нужно адаптировать.
# googlenet_pretrained.aux1.fc2 = nn.Linear(googlenet_pretrained.aux1.fc2.in_features, num_classes)
# googlenet_pretrained.aux2.fc2 = nn.Linear(googlenet_pretrained.aux2.fc2.in_features, num_classes)


num_ftrs_scratch_googlenet = googlenet_scratch.fc.in_features
googlenet_scratch.fc = nn.Linear(num_ftrs_scratch_googlenet, num_classes)
# googlenet_scratch.aux1.fc2 = nn.Linear(googlenet_scratch.aux1.fc2.in_features, num_classes)
# googlenet_scratch.aux2.fc2 = nn.Linear(googlenet_scratch.aux2.fc2.in_features, num_classes)


# Пример дообучения (Fine-tuning Setup - Концептуально):
# model_to_train = googlenet_pretrained.to(device)
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model_to_train.parameters(), lr=1e-4)
# # При обучении GoogLeNet часто считают суммарные потери от основного и вспомогательных выходов
# # В цикле обучения:
# # if model_to_train.training: # Если в режиме обучения
# #     outputs, aux1_outputs, aux2_outputs = model_to_train(inputs)
# #     loss1 = criterion(outputs, labels)
# #     loss2 = criterion(aux1_outputs, labels)
# #     loss3 = criterion(aux2_outputs, labels)
# #     loss = loss1 + 0.3 * loss2 + 0.3 * loss3 # Взвешенная сумма
# # else: # В режиме model.eval()
# #     outputs = model_to_train(inputs)
# #     loss = criterion(outputs, labels)
# # loss.backward()
# # optimizer.step()
# # Далее стандартный цикл обучения...

# --------------------------------------------------

# Блок 6: DenseNet (Densely Connected Convolutional Network)

# Краткое описание:
# Каждый слой получает на вход карты признаков всех предыдущих слоев (внутри одного "dense block").
# Это способствует лучшему потоку градиентов и повторному использованию признаков.
# Очень эффективна по параметрам (меньше параметров, чем ResNet при схожей точности).
# Варианты: DenseNet121, DenseNet169, DenseNet201, DenseNet161.

# Загрузка модели:
# Загрузить предобученную DenseNet121
densenet121_pretrained = models.densenet121(pretrained=True)
# Загрузить только архитектуру DenseNet121
densenet121_scratch = models.densenet121(pretrained=False)

# Модификация для своих данных:
# У DenseNet последний слой называется 'classifier'.
num_classes = 102 # Пример: Oxford Flowers 102
num_ftrs_densenet = densenet121_pretrained.classifier.in_features
densenet121_pretrained.classifier = nn.Linear(num_ftrs_densenet, num_classes)

num_ftrs_scratch_densenet = densenet121_scratch.classifier.in_features
densenet121_scratch.classifier = nn.Linear(num_ftrs_scratch_densenet, num_classes)

# Пример дообучения (Fine-tuning Setup - Концептуально):
# model_to_train = densenet121_pretrained.to(device)
# criterion = nn.CrossEntropyLoss()
# # Оптимизация только классификатора
# # optimizer = optim.Adam(model_to_train.classifier.parameters(), lr=0.001)
# # Оптимизация всей сети (с малым lr)
# optimizer = optim.Adam(model_to_train.parameters(), lr=1e-4)
# # Далее стандартный цикл обучения...

# --------------------------------------------------

# Блок 7: MobileNet

# Краткое описание:
# Семейство легких и быстрых архитектур, оптимизированных для мобильных устройств и встраиваемых систем.
# Используют "depthwise separable convolutions" для значительного снижения количества вычислений и параметров.
# Версии: MobileNetV2, MobileNetV3 (Small/Large).

# Загрузка модели:
# Загрузить предобученную MobileNetV2
mobilenet_v2_pretrained = models.mobilenet_v2(pretrained=True)
# Загрузить только архитектуру MobileNetV2
mobilenet_v2_scratch = models.mobilenet_v2(pretrained=False)
# Аналогично для MobileNetV3
# mobilenet_v3_large_pretrained = models.mobilenet_v3_large(pretrained=True)

# Модификация для своих данных:
# У MobileNetV2/V3 последний блок называется 'classifier'. Обычно это Sequential с Dropout и Linear.
num_classes = 2 # Пример: Бинарная классификация (cat/dog)
# Для MobileNetV2
num_ftrs_mobilenet_v2 = mobilenet_v2_pretrained.classifier[1].in_features
mobilenet_v2_pretrained.classifier[1] = nn.Linear(num_ftrs_mobilenet_v2, num_classes)
# Для MobileNetV3 Large (структура classifier может немного отличаться)
# num_ftrs_mobilenet_v3 = mobilenet_v3_large_pretrained.classifier[3].in_features
# mobilenet_v3_large_pretrained.classifier[3] = nn.Linear(num_ftrs_mobilenet_v3, num_classes)

# То же для scratch моделей
num_ftrs_scratch_mobilenet_v2 = mobilenet_v2_scratch.classifier[1].in_features
mobilenet_v2_scratch.classifier[1] = nn.Linear(num_ftrs_scratch_mobilenet_v2, num_classes)

# Пример дообучения (Fine-tuning Setup - Концептуально):
# model_to_train = mobilenet_v2_pretrained.to(device)
# criterion = nn.CrossEntropyLoss() # или nn.BCEWithLogitsLoss для бинарной
# optimizer = optim.Adam(model_to_train.parameters(), lr=1e-4)
# # Далее стандартный цикл обучения...

# --------------------------------------------------

# Блок 8: EfficientNet

# Краткое описание:
# Семейство моделей, достигающих state-of-the-art точности при значительно меньшем количестве параметров и FLOPS.
# Используют метод "compound scaling", который оптимально масштабирует глубину, ширину и разрешение сети одновременно.
# Начинается с базовой сети EfficientNet-B0 и масштабируется до B7.
# `torchvision` предоставляет B0-B7.

# Загрузка модели:
# Загрузить предобученную EfficientNet-B0
efficientnet_b0_pretrained = models.efficientnet_b0(pretrained=True)
# Загрузить только архитектуру EfficientNet-B0
efficientnet_b0_scratch = models.efficientnet_b0(pretrained=False)

# Модификация для своих данных:
# У EfficientNet последний блок называется 'classifier', содержащий Dropout и Linear слой.
num_classes = 1000 # Пример: ImageNet
num_ftrs_efficientnet = efficientnet_b0_pretrained.classifier[1].in_features
efficientnet_b0_pretrained.classifier[1] = nn.Linear(num_ftrs_efficientnet, num_classes)

num_ftrs_scratch_efficientnet = efficientnet_b0_scratch.classifier[1].in_features
efficientnet_b0_scratch.classifier[1] = nn.Linear(num_ftrs_scratch_efficientnet, num_classes)

# Пример дообучения (Fine-tuning Setup - Концептуально):
# model_to_train = efficientnet_b0_pretrained.to(device)
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model_to_train.parameters(), lr=1e-4) # EfficientNet часто обучают с RMSprop или AdamW
# # Далее стандартный цикл обучения...

# --------------------------------------------------

# Блок 9: Vision Transformer (ViT)

# Краткое описание:
# Адаптация архитектуры Transformer (изначально из NLP) для задач CV.
# Делит изображение на патчи (patches), линейно их встраивает (embeds), добавляет позиционные эмбеддинги
# и подает последовательность патчей в стандартный Transformer Encoder.
# Для классификации используется специальный токен [CLS] или усреднение выходов.
# Требует больших датасетов для обучения с нуля, но хорошо работает с transfer learning.
# `torchvision` предоставляет несколько вариантов ViT.

# Загрузка модели:
# Загрузить предобученную ViT B/16 (Base модель, размер патча 16x16)
vit_b16_pretrained = models.vit_b_16(pretrained=True)
# Загрузить только архитектуру ViT B/16
vit_b16_scratch = models.vit_b_16(pretrained=False)

# Модификация для своих данных:
# У ViT классификационная голова называется 'heads'. Обычно это один Linear слой.
num_classes = 37 # Пример: Oxford-IIIT Pet Dataset
num_ftrs_vit = vit_b16_pretrained.heads.head.in_features # Доступ к слою 'head' внутри 'heads'
vit_b16_pretrained.heads.head = nn.Linear(num_ftrs_vit, num_classes)

num_ftrs_scratch_vit = vit_b16_scratch.heads.head.in_features
vit_b16_scratch.heads.head = nn.Linear(num_ftrs_scratch_vit, num_classes)

# Пример дообучения (Fine-tuning Setup - Концептуально):
# model_to_train = vit_b16_pretrained.to(device)
# criterion = nn.CrossEntropyLoss()
# # ViT часто требует специфических настроек оптимизатора (AdamW) и расписания lr
# optimizer = optim.AdamW(model_to_train.parameters(), lr=1e-5, weight_decay=0.01)
# # Далее стандартный цикл обучения...

# --------------------------------------------------

# Общие Замечания по Дообучению:
# 1. Предобработка данных: Убедитесь, что ваши данные предобработаны так же,
#    как данные, на которых обучалась предобученная модель (особенно нормализация).
#    `torchvision` обычно предоставляет нужные параметры `mean` и `std`.
# 2. Learning Rate: При fine-tuning всей сети используйте значительно меньший learning rate
#    (например, 1e-4, 1e-5), чем при обучении с нуля, чтобы не разрушить предобученные веса.
# 3. Оптимизатор: Adam, AdamW, SGD с моментумом - частый выбор. Иногда RMSprop.
# 4. Заморозка слоев: Для feature extraction заморозьте параметры с помощью `param.requires_grad = False`.
#    Не забудьте передать в оптимизатор только размороженные параметры.
# 5. Разморозка слоев: При full fine-tuning можно постепенно размораживать слои,
#    начиная с верхних (ближе к выходу) и двигаясь к нижним, с разными learning rates.
# 6. `model.train()` и `model.eval()`: Не забывайте переключать режимы модели.
#    `eval()` важен для отключения Dropout и использования статистики Batch Normalization,
#    накопленной во время обучения.

# --------------------------------------------------

In [ ]:
# Блок 1: Общая Подготовка и Контекст

# Импорты
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader, TensorDataset # TensorDataset для примера
import time
import copy # Для копирования состояния модели
import os # Для создания директорий

# Определение устройства (GPU или CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Параметры (Примеры)
num_classes = 10 # Количество классов в вашем датасете (например, CIFAR-10)
batch_size = 32
learning_rate_fe = 0.001 # Learning rate для Feature Extraction
learning_rate_ft = 0.0001 # Learning rate для Fine-tuning (обычно ниже)
num_epochs = 5 # Количество эпох для обучения (в реальной задаче нужно больше)
# Путь для сохранения моделей
model_save_dir = "./saved_models"
os.makedirs(model_save_dir, exist_ok=True)


# --- Создание Фиктивных Данных для Демонстрации ---
# В реальной задаче здесь будут ваши DataLoader'ы с реальными данными
# Используем случайные тензоры как имитацию изображений и меток
# ResNet обычно ожидает вход 224x224
print("Creating dummy data...")
dummy_data = torch.randn(batch_size * 10, 3, 224, 224) # Увеличим немного размер
dummy_labels = torch.randint(0, num_classes, (batch_size * 10,))
dummy_dataset = TensorDataset(dummy_data, dummy_labels)
# Разделим на train/val условно
train_size = int(0.8 * len(dummy_dataset))
val_size = len(dummy_dataset) - train_size
# Используем генератор для воспроизводимости разделения
generator = torch.Generator().manual_seed(42)
dummy_train_dataset, dummy_val_dataset = torch.utils.data.random_split(dummy_dataset, [train_size, val_size], generator=generator)

dummy_train_loader = DataLoader(dummy_train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
dummy_val_loader = DataLoader(dummy_val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# Словарь с загрузчиками данных
dataloaders = {'train': dummy_train_loader, 'val': dummy_val_loader}
dataset_sizes = {'train': len(dummy_train_dataset), 'val': len(dummy_val_dataset)}
print("Dummy data created.")

# Функция для обучения модели (общая для обоих подходов)
# Она будет вызываться с разной конфигурацией модели и оптимизатора
def train_model(model, criterion, optimizer, scheduler=None, num_epochs=5, model_name="model"):
    since = time.time()
    # Сохраняем лучшие веса модели
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

    print(f"\nStarting training for {model_name}...")
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Каждая эпоха имеет фазу обучения и валидации
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Установить модель в режим обучения
            else:
                model.eval()   # Установить модель в режим оценки

            running_loss = 0.0
            running_corrects = 0

            # Итерация по данным
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Обнулить градиенты параметра
                optimizer.zero_grad()

                # Прямой проход
                # Отслеживать историю только в режиме обучения
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Обратный проход + оптимизация только в фазе обучения
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Статистика
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            history[f'{phase}_loss'].append(epoch_loss)
            history[f'{phase}_acc'].append(epoch_acc.item()) # Сохраняем как float

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Глубокое копирование модели, если достигнута лучшая точность на валидации
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                # Сохраняем лучшую модель
                torch.save(model.state_dict(), os.path.join(model_save_dir, f'{model_name}_best.pth'))
                print(f"Saved best model weights to {os.path.join(model_save_dir, f'{model_name}_best.pth')}")


        # Шаг планировщика LR (если используется)
        if phase == 'train' and scheduler:
             scheduler.step() # Некоторые планировщики требуют loss на шаге (ReduceLROnPlateau)

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # Загрузить лучшие веса модели
    model.load_state_dict(best_model_wts)
    return model, history

# --------------------------------------------------

# Блок 2: Feature Extraction (Извлечение Признаков)

# Концепция:
# 1. Загружаем предобученную модель ResNet (например, ResNet18 или ResNet50).
# 2. "Замораживаем" веса всех слоев модели, кроме последнего полносвязного слоя (классификатора).
#    Это означает, что во время обучения будут обновляться только веса этого последнего слоя.
#    Сверточные слои используются как фиксированный экстрактор признаков.
# 3. Заменяем последний слой на новый, соответствующий количеству классов в нашем датасете.
# 4. Обучаем модель. Обновляются только веса нового классификатора.

print("\n--- Feature Extraction ---")

# 1. Загрузка предобученной модели ResNet18
model_fe = models.resnet18(pretrained=True)

# 2. Заморозка весов всех слоев
for param in model_fe.parameters():
    param.requires_grad = False # Отключаем расчет градиентов для всех существующих параметров

# 3. Замена последнего слоя (fc - fully connected)
num_ftrs_fe = model_fe.fc.in_features # Получаем количество входов последнего слоя
# Создаем новый слой. По умолчанию requires_grad=True для новых слоев.
model_fe.fc = nn.Linear(num_ftrs_fe, num_classes)

# Перемещаем модель на нужное устройство (GPU/CPU)
model_fe = model_fe.to(device)

# Убедимся, что только параметры последнего слоя будут обновляться
print("Params to learn in Feature Extraction:")
params_to_update_fe = []
for name, param in model_fe.named_parameters():
    if param.requires_grad == True:
        params_to_update_fe.append(param)
        print("\t", name)

# 4. Определение функции потерь и оптимизатора
criterion_fe = nn.CrossEntropyLoss()
# В оптимизатор передаем ТОЛЬКО параметры, которые нужно обучать (параметры нового fc слоя)
optimizer_fe = optim.Adam(params_to_update_fe, lr=learning_rate_fe)

# Запуск обучения (используем общую функцию train_model)
# Раскомментируйте для запуска обучения
model_fe_trained, history_fe = train_model(model_fe, criterion_fe, optimizer_fe,
                                            num_epochs=num_epochs, model_name="resnet18_fe")
print("Feature Extraction Training Finished.")

# После обучения модель model_fe_trained готова к использованию
# или может служить инициализацией для Fine-tuning.

# --------------------------------------------------

# Блок 3: Fine-tuning (Дообучение)

# Концепция:
# 1. Загружаем предобученную модель ResNet.
# 2. Заменяем последний слой на новый, соответствующий нашему количеству классов.
# 3. "Размораживаем" все слои модели (или часть из них, например, несколько последних блоков ResNet).
#    Это означает, что градиенты будут рассчитываться для всех (или выбранных) параметров.
# 4. Обучаем всю модель на новом датасете, но с очень маленькой скоростью обучения (learning rate).
#    Это позволяет "тонко настроить" предобученные веса под специфику новых данных,
#    не разрушая при этом полезные признаки, изученные на ImageNet.

print("\n--- Fine-tuning ---")

# 1. Загрузка предобученной модели ResNet18
# Можно начать с нуля или взять модель после Feature Extraction
model_ft = models.resnet18(pretrained=True)
# Если хотите начать с модели после FE:
# model_ft = model_fe_trained # Загрузить уже обученную модель FE
# Или загрузить сохраненные веса FE:
# model_ft = models.resnet18(pretrained=False) # Загрузить архитектуру
# num_ftrs_ft_load = model_ft.fc.in_features
# model_ft.fc = nn.Linear(num_ftrs_ft_load, num_classes) # Адаптировать fc слой
# fe_weights_path = os.path.join(model_save_dir, 'resnet18_fe_best.pth')
# if os.path.exists(fe_weights_path):
#     model_ft.load_state_dict(torch.load(fe_weights_path))
#     print(f"Loaded weights from {fe_weights_path}")
# else:
#     print("FE weights not found, starting fine-tuning from ImageNet weights.")
#     model_ft = models.resnet18(pretrained=True) # Начать с ImageNet, если FE веса не найдены

# 2. Замена последнего слоя (если не загрузили модель после FE)
num_ftrs_ft = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs_ft, num_classes)

# 3. Разморозка всех слоев (по умолчанию они разморожены, если не замораживали ранее)
# Убедимся, что requires_grad = True для всех параметров
for param in model_ft.parameters():
     param.requires_grad = True

# Перемещаем модель на устройство
model_ft = model_ft.to(device)

# Выведем параметры для обучения (должны быть все параметры модели)
print("Params to learn in Fine-tuning:")
params_to_update_ft = []
total_params = 0
for name, param in model_ft.named_parameters():
    if param.requires_grad == True:
        params_to_update_ft.append(param)
        total_params += param.numel() # Считаем общее количество обучаемых параметров
        # print("\t", name) # Раскомментируйте, если хотите увидеть все параметры
print(f"\tTotal trainable parameters: {total_params}")


# 4. Определение функции потерь и оптимизатора
criterion_ft = nn.CrossEntropyLoss()
# В оптимизатор передаем ВСЕ параметры модели, но с МАЛЕНЬКИМ learning rate
optimizer_ft = optim.Adam(params_to_update_ft, lr=learning_rate_ft) # Используем lr для fine-tuning
# Часто используют SGD с моментумом для fine-tuning
# optimizer_ft = optim.SGD(params_to_update_ft, lr=learning_rate_ft, momentum=0.9)

# (Опционально) Добавление планировщика скорости обучения (Learning Rate Scheduler)
# Уменьшает LR во время обучения, что часто улучшает сходимость
from torch.optim import lr_scheduler
# Уменьшать LR в gamma раз каждые step_size эпох
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=3, gamma=0.1)

# Запуск обучения
# Раскомментируйте для запуска обучения
# model_ft_trained, history_ft = train_model(model_ft, criterion_ft, optimizer_ft,
#                                            scheduler=exp_lr_scheduler, # Передаем планировщик
#                                            num_epochs=num_epochs, model_name="resnet18_ft")
# print("Fine-tuning Training Finished.")

# Модель model_ft_trained готова к использованию.

# --------------------------------------------------

# Блок 4: Важные Замечания (Продолжение)

# 2. Выбор между Feature Extraction и Fine-tuning:
#    - Feature Extraction:
#      - Быстрее обучается, требует меньше вычислительных ресурсов.
#      - Хороший выбор, если ваш датасет мал и/или очень похож на ImageNet.
#      - Меньше риск переобучения на маленьких датасетах.
#    - Fine-tuning:
#      - Потенциально может дать более высокую точность, так как адаптирует больше весов.
#      - Требует больше времени и ресурсов.
#      - Лучше подходит для больших датасетов или датасетов, которые отличаются от ImageNet.
#      - Требует аккуратного выбора learning rate, чтобы не испортить предобученные веса.
#    - Часто начинают с Feature Extraction, а затем делают Fine-tuning, используя
#      веса, полученные после Feature Extraction, как начальные.

# 3. Learning Rate Schedulers (Планировщики Скорости Обучения):
#    - Уменьшение learning rate во время обучения (learning rate decay) часто помогает
#      модели лучше сойтись к хорошему минимуму функции потерь.
#    - `torch.optim.lr_scheduler` предоставляет разные стратегии:
#      - `StepLR`: Уменьшает LR в `gamma` раз каждые `step_size` эпох.
#      - `MultiStepLR`: Уменьшает LR в `gamma` раз на заданных эпохах (`milestones`).
#      - `ExponentialLR`: Умножает LR на `gamma` каждую эпоху.
#      - `ReduceLROnPlateau`: Уменьшает LR, когда метрика перестает улучшаться (например, val_loss).
#    - Планировщик вызывается после шага оптимизатора в каждой эпохе (`scheduler.step()`).

# 4. Дифференциальные Learning Rates:
#    - При Fine-tuning иногда полезно задавать разные learning rates для разных частей сети.
#    - Например, можно установить очень маленький LR для замороженных слоев (backbone)
#      и больший LR для нового классификатора (head).
#    - Это можно сделать, передав в оптимизатор список словарей с параметрами:
#      ```python
#      # Пример (не запускается здесь)
#      # optimizer = optim.Adam([
#      #     {'params': model.conv1.parameters(), 'lr': learning_rate_ft * 0.1}, # Меньший LR для ранних слоев
#      #     {'params': model.layer1.parameters(), 'lr': learning_rate_ft * 0.1},
#      #     # ... другие слои backbone ...
#      #     {'params': model.fc.parameters(), 'lr': learning_rate_ft} # Больший LR для головы
#      # ], lr=learning_rate_ft) # lr по умолчанию для параметров не в группах
#      ```

# 5. Сохранение и Загрузка Моделей:
#    - Важно сохранять веса модели во время обучения (особенно лучшие веса на валидации).
#    - `torch.save(model.state_dict(), PATH)`: Сохраняет только параметры модели (рекомендуется).
#    - `torch.save(model, PATH)`: Сохраняет всю модель (менее гибко).
#    - Для загрузки весов:
#      ```python
#      # model = TheModelClass(*args, **kwargs) # Сначала создайте экземпляр модели
#      # model.load_state_dict(torch.load(PATH))
#      # model.eval() # Перевести в режим оценки перед использованием
#      ```

# 6. Оценка Модели:
#    - После обучения необходимо оценить финальную модель на отдельной тестовой выборке
#      (которая не использовалась ни для обучения, ни для валидации/выбора лучших весов).
#    - Используйте `model.eval()` и `with torch.no_grad():` для оценки.
#    - Рассчитайте метрики (Accuracy, Precision, Recall, F1-score, Confusion Matrix).

# --------------------------------------------------

In [ ]:
# Блок 1: Проблема Разной Размерности Входных Изображений

# Контекст:
# Большинство популярных моделей в `torchvision.models` (ResNet, VGG, EfficientNet и т.д.)
# были предобучены на датасете ImageNet.
# ImageNet содержит изображения, которые обычно приводятся к размеру 224x224 или 299x299 пикселей
# перед подачей в модель во время её оригинального обучения.

# Проблема:
# Ваши реальные данные могут иметь совершенно другие размеры (например, 640x480, 1024x768, или даже разные размеры для разных изображений).
# Что произойдет, если подать изображение другого размера в предобученную модель?

# Почему возникает ошибка (или неоптимальная работа)?
# 1. Сверточные слои (Conv2d): Сами по себе относительно гибки к размеру входа. Они оперируют локально.
#    Изменение размера входа изменит размер выходных карт признаков.
# 2. Пулинг слои (MaxPool2d, AvgPool2d): Также гибки, уменьшают пространственный размер карт признаков.
# 3. Adaptive Pooling (nn.AdaptiveAvgPool2d, nn.AdaptiveMaxPool2d): Ключевой элемент!
#    Многие современные архитектуры (ResNet, DenseNet, EfficientNet, MobileNetV2/V3) используют
#    адаптивный пулинг перед последним полносвязным слоем.
#    Например, `nn.AdaptiveAvgPool2d((1, 1))` возьмет карту признаков ЛЮБОГО пространственного размера (H x W)
#    и выдаст карту фиксированного размера (1 x 1), сохраняя глубину каналов.
#    Это делает модель более устойчивой к разным размерам входа *до* полносвязного слоя.
# 4. Полносвязные слои (Linear): Вот здесь основная проблема!
#    Слой `nn.Linear(in_features, out_features)` ожидает на вход вектор строго определенной длины `in_features`.
#    Эта длина `in_features` в предобученной модели была рассчитана исходя из ожидаемого размера карт признаков
#    после сверточных/пулинг слоев (часто после адаптивного пулинга или flatten), который, в свою очередь,
#    зависел от исходного размера входного изображения (например, 224x224).
#    Если подать изображение другого размера, то размер вектора перед `nn.Linear` может измениться
#    (особенно если нет адаптивного пулинга), что вызовет ошибку несоответствия размерности (size mismatch error).

# --------------------------------------------------

# Блок 2: Решение 1 - Трансформация Входных Изображений (Самый Распространенный и Рекомендуемый)

# Идея:
# Привести все ваши входные изображения к тому размеру, который ожидала модель при обучении на ImageNet,
# используя трансформации из `torchvision.transforms`.

# Как реализовать:
import torchvision.transforms as transforms

# Размер, ожидаемый моделью (например, для ResNet, VGG, MobileNetV2)
input_size = 224

# Трансформации для Обучения (включают аугментацию и изменение размера)
# RandomResizedCrop вырезает случайную часть изображения и изменяет её размер до input_size.
# Это хорошая аугментация и способ обработки разных размеров/соотношений сторон.
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(input_size), # Вырезать и изменить размер
    transforms.RandomHorizontalFlip(),      # Случайное горизонтальное отражение
    transforms.ToTensor(),                  # Конвертировать в PyTorch тензор ([0, 1])
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Нормализация ImageNet
])

# Трансформации для Валидации/Тестирования/Инференса (без случайных аугментаций)
# Обычно сначала изменяют размер меньшей стороны до чуть большего значения (e.g., 256),
# а затем вырезают центральную часть нужного размера (e.g., 224).
val_test_transforms = transforms.Compose([
    transforms.Resize(256),                 # Изменить размер меньшей стороны до 256
    transforms.CenterCrop(input_size),      # Вырезать центральный квадрат 224x224
    transforms.ToTensor(),                  # Конвертировать в PyTorch тензор
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Нормализация ImageNet
])

# Применение:
# Эти объекты `transforms` передаются в `torchvision.datasets.ImageFolder`
# или применяются вручную к вашим изображениям перед подачей в модель.
# train_dataset = datasets.ImageFolder('path/to/train', transform=train_transforms)
# val_dataset = datasets.ImageFolder('path/to/val', transform=val_test_transforms)
# train_loader = DataLoader(train_dataset, ...)
# val_loader = DataLoader(val_dataset, ...)

# Преимущества:
# + Простота: Не требует изменения архитектуры модели.
# + Эффективность: Позволяет напрямую использовать предобученные веса ImageNet.
# + Стандартный подход: Широко используется и хорошо работает на практике.
# + Совместимость: Работает с любой предобученной моделью из torchvision.

# Недостатки:
# - Потеря информации: Изменение размера может исказить соотношение сторон. Обрезка (cropping) может удалить важные части изображения, если объект находится у края или занимает не центр кадра.
# - `RandomResizedCrop` во время обучения помогает модели стать более устойчивой к этому.

# --------------------------------------------------

# Блок 3: Решение 2 - Модификация Архитектуры Модели (Менее Распространено для Классификации)

# Идея:
# Адаптировать архитектуру модели так, чтобы она могла принимать изображения разного размера.
# Это обычно включает замену или модификацию слоев перед полносвязным классификатором.

# Вариант А: Использование Моделей с Adaptive Pooling (Большинство современных моделей)
# Если модель УЖЕ использует `nn.AdaptiveAvgPool2d((1, 1))` или `nn.AdaptiveMaxPool2d((1, 1))`
# перед последним `nn.Linear` слоем (как в ResNet, DenseNet, EfficientNet, MobileNetV2/V3),
# то она *уже* может обрабатывать разные пространственные размеры карт признаков *до* этого пулинга.
# Проблема несоответствия размера для `nn.Linear` все еще может возникнуть, если
# количество каналов перед адаптивным пулингом не совпадает с ожидаемым `in_features`
# (хотя обычно это не так при использовании стандартных архитектур).
# Главное, что нужно сделать в этом случае - это заменить последний `nn.Linear` слой,
# чтобы он соответствовал ВАШЕМУ количеству классов (как мы делали в примерах FE/FT).
# Подача изображений другого размера *может* сработать без ошибки, но результаты могут быть
# неоптимальными, так как сверточные слои будут извлекать признаки из другого масштаба/разрешения,
# чем тот, на котором они обучались.

# Вариант Б: Добавление/Замена на Adaptive Pooling (Если его нет)
# Если у вас старая модель или кастомная архитектура без адаптивного пулинга,
# вы можете вручную заменить последний слой MaxPool/AvgPool + Flatten на `nn.AdaptiveAvgPool2d((1, 1))` + `nn.Flatten(1)`.
import torch.nn as nn
import torchvision.models as models

# # Пример (гипотетический, для модели без адаптивного пулинга):
# model = models.vgg16(pretrained=True) # VGG использует адаптивный пулинг в `classifier` блоке, но представим что нет
# # Представим, что последние слои были:
# # model.features -> nn.MaxPool2d -> nn.Flatten -> nn.Linear
# # Мы можем заменить их:
# model.avgpool = nn.AdaptiveAvgPool2d((7, 7)) # Стандартный для VGG перед классификатором
# # Классификатор VGG уже ожидает выход avgpool (7x7), так что здесь замена не нужна,
# # но если бы его не было, мы бы добавили AdaptiveAvgPool2d((1, 1))
# # и соответствующим образом изменили бы in_features первого Linear слоя.

# # Главное, что нужно сделать - заменить классификатор под свои классы:
# num_classes = 10 # Ваш пример
# num_ftrs_vgg = model.classifier[6].in_features
# model.classifier[6] = nn.Linear(num_ftrs_vgg, num_classes)

# Преимущества:
# + Потенциальное сохранение информации: Не происходит обрезки или искажения исходного изображения.
# + Гибкость: Модель может обрабатывать разные размеры на лету.

# Недостатки:
# - Сложность: Требует модификации архитектуры модели.
# - Неоптимальные признаки: Сверточные слои, обученные на признаках из изображений 224x224, могут работать хуже при извлечении признаков из изображений значительно другого размера. Результат не гарантирован.
# - Необходимость Fine-tuning: Почти наверняка потребуется дообучение (fine-tuning) модели на ваших данных с новыми размерами, чтобы адаптировать веса.
# - Не всегда нужно: Если модель уже имеет адаптивный пулинг, основная проблема решается им, и остается только вопрос оптимальности признаков.

# --------------------------------------------------

# Блок 4: Решение 3 - Патчинг / Скользящее Окно (Редко для Классификации, Чаще для Детекции/Сегментации)

# Идея:
# Если у вас очень большие изображения и важно сохранить высокое разрешение:
# 1. Разделить большое входное изображение на несколько перекрывающихся (или неперекрывающихся) патчей (участков) размера, ожидаемого моделью (например, 224x224).
# 2. Прогнать каждый патч через предобученную модель (после замены классификатора).
# 3. Агрегировать результаты (например, усреднить предсказания, взять максимум или использовать более сложную логику) для получения финального предсказания для всего изображения.

# Преимущества:
# + Сохранение деталей: Работает с исходным разрешением по частям.

# Недостатки:
# - Вычислительно дорого: Модель применяется много раз для одного изображения.
# - Сложная реализация: Требует логики нарезки на патчи и агрегации результатов.
# - Избыточно для классификации: Обычно для классификации всего изображения достаточно информации из измененного/обрезанного изображения (Решение 1). Этот подход более актуален для задач, где важна локализация (детекция, сегментация).

# --------------------------------------------------

# Блок 5: Вывод и Рекомендации

# 1.  **Начните с Решения 1 (Трансформация Входа):** Это самый простой, стандартный и часто наиболее эффективный способ использования предобученных моделей для классификации с данными нестандартного размера. Используйте `transforms.RandomResizedCrop` для обучения и `transforms.Resize` + `transforms.CenterCrop` для валидации/тестирования. Не забудьте про `transforms.Normalize` с параметрами ImageNet.

# 2.  **Замените Последний Слой:** Независимо от того, как вы решаете проблему размера, вам *всегда* нужно заменить последний полносвязный слой (`fc` у ResNet, `classifier` у VGG/DenseNet/MobileNet/EfficientNet, `heads.head` у ViT) на новый `nn.Linear`, у которого `out_features` равно количеству классов в вашем датасете.

# 3.  **Рассмотрите Fine-tuning:** Даже если вы используете Решение 1, дообучение (fine-tuning) модели на ваших данных (с правильной предобработкой) обычно приводит к лучшим результатам, чем простое использование модели "как есть" (даже после замены классификатора).

# 4.  **Модификация Архитектуры (Решение 2):** Используйте с осторожностью. Если ваша модель уже имеет адаптивный пулинг, она технически может справиться с разными размерами до FC слоя, но качество признаков может пострадать. Если адаптивного пулинга нет, его добавление может помочь, но все равно потребуется тщательное дообучение.

# 5.  **Патчинг (Решение 3):** Обычно избыточен для стандартной задачи классификации изображений.

# В подавляющем большинстве случаев для задач классификации с использованием предобученных моделей torchvision, **изменение размера входных изображений с помощью `torchvision.transforms` является предпочтительным методом.**
# --------------------------------------------------

In [ ]:
# Блок 1: Введение в Обнаружение Объектов (Object Detection)

# Что такое Обнаружение Объектов?
# Это задача Computer Vision, которая заключается не только в классификации объектов на изображении,
# но и в определении их точного местоположения с помощью ограничивающих рамок (bounding boxes).
# Вход: Изображение.
# Выход: Список обнаруженных объектов, где каждый элемент содержит:
#   - Координаты ограничивающей рамки (bounding box), обычно [xmin, ymin, xmax, ymax].
#   - Метку класса объекта (например, "человек", "автомобиль", "собака").
#   - Оценку уверенности (confidence score) для каждого обнаружения.

# Цель Обнаружения Объектов:
# Научить модель "видеть", где находятся объекты разных классов на изображении.

# Отличие от Классификации Изображений:
# - Классификация: Что изображено на картинке в целом? (1 метка на изображение)
# - Обнаружение: Что изображено и где именно? (Много меток + координаты рамок на изображение)

# Отличие от Сегментации Изображений:
# - Обнаружение: Прямоугольные рамки вокруг объектов.
# - Сегментация: Попиксельная маска для каждого объекта (более точная локализация формы).

# --------------------------------------------------

# Блок 2: Ключевые Концепции

# 1. Ограничивающая Рамка (Bounding Box):
#    - Прямоугольник, описывающий положение объекта.
#    - Форматы представления:
#      - `[xmin, ymin, xmax, ymax]`: Координаты левого верхнего и правого нижнего углов. (Часто используется в PyTorch/torchvision)
#      - `[x_center, y_center, width, height]`: Координаты центра, ширина и высота. (Часто используется в YOLO)
#    - Координаты обычно нормализованы (от 0 до 1) или абсолютные (в пикселях). `torchvision` обычно работает с абсолютными пиксельными координатами.

# 2. Intersection over Union (IoU):
#    - Метрика для оценки того, насколько хорошо предсказанная рамка (`pred_box`) совпадает с истинной рамкой (`gt_box`).
#    - Рассчитывается как площадь пересечения рамок, деленная на площадь их объединения.
#    - IoU = Area(Intersection) / Area(Union)
#    - Значение от 0 (нет пересечения) до 1 (идеальное совпадение).
#    - Используется для определения, является ли предсказание "правильным" (True Positive) во время оценки (например, если IoU > 0.5).

def calculate_iou(boxA, boxB):
    # box format: [xmin, ymin, xmax, ymax]
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = max(0, xB - xA) * max(0, yB - yA)

    # compute the area of both the prediction and ground-truth rectangles
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the intersection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

# 3. Non-Maximum Suppression (NMS):
#    - Пост-обработка для удаления избыточных, сильно перекрывающихся рамок для одного и того же объекта.
#    - Алгоритм:
#      1. Отсортировать все предсказанные рамки по убыванию уверенности (confidence score).
#      2. Выбрать рамку с наивысшей уверенностью и добавить ее в финальный список.
#      3. Удалить все остальные рамки, которые сильно перекрываются с выбранной (IoU > порога NMS, например, 0.45).
#      4. Повторять шаги 2-3, пока не останется рамок.
#    - `torchvision.ops.nms()` предоставляет реализацию NMS.

# 4. Mean Average Precision (mAP):
#    - Стандартная метрика для оценки качества моделей обнаружения объектов.
#    - Рассчитывается на основе кривой Precision-Recall для каждого класса.
#    - Сначала вычисляется Average Precision (AP) для каждого класса (площадь под кривой Precision-Recall).
#    - Затем AP усредняется по всем классам для получения mAP.
#    - Часто указывается с порогом IoU (например, mAP@0.5, mAP@0.75, mAP@[0.5:0.95]).

# 5. Подходы к Обнаружению:
#    - Двухэтапные (Two-stage) детекторы:
#      - Сначала генерируют "предложения" регионов (Region Proposals), где могут быть объекты (например, с помощью Region Proposal Network - RPN).
#      - Затем классифицируют объекты в этих регионах и уточняют их рамки.
#      - Примеры: R-CNN, Fast R-CNN, Faster R-CNN, Mask R-CNN.
#      - Обычно более точные, но медленнее.
#    - Одноэтапные (One-stage) детекторы:
#      - Предсказывают классы и координаты рамок напрямую из карт признаков за один проход.
#      - Примеры: YOLO (You Only Look Once), SSD (Single Shot MultiBox Detector), RetinaNet, FCOS.
#      - Обычно быстрее, но могут быть менее точными (хотя современные версии очень конкурентоспособны).

# --------------------------------------------------

# Блок 3: Модели Обнаружения в `torchvision.models.detection`

# `torchvision` предоставляет готовые реализации популярных моделей обнаружения,
# включая предобученные на датасете COCO (Common Objects in Context).

# 1. Faster R-CNN:
#    - Классический и мощный двухэтапный детектор.
#    - Использует ResNet (или другие сети) как backbone для извлечения признаков.
#    - Включает Region Proposal Network (RPN) для генерации кандидатов.
#    - Использует RoIAlign (улучшенный RoIPooling) для извлечения признаков из регионов.
#    - Имеет отдельные "головы" для классификации и регрессии рамок.
#    - Пример загрузки: `models.detection.fasterrcnn_resnet50_fpn(pretrained=True)`
#      - `fpn` (Feature Pyramid Network) улучшает обнаружение объектов разного масштаба.

# 2. Mask R-CNN:
#    - Расширение Faster R-CNN, которое добавляет ветвь для предсказания маски сегментации для каждого объекта.
#    - Может использоваться и для обнаружения (просто игнорируя выход маски).
#    - Пример загрузки: `models.detection.maskrcnn_resnet50_fpn(pretrained=True)`

# 3. RetinaNet:
#    - Популярный одноэтапный детектор.
#    - Использует Focal Loss для решения проблемы сильного дисбаланса между фоновыми и объектовыми примерами во время обучения.
#    - Пример загрузки: `models.detection.retinanet_resnet50_fpn(pretrained=True)`

# 4. SSD (Single Shot MultiBox Detector):
#    - Еще один эффективный одноэтапный детектор.
#    - Делает предсказания на разных уровнях карт признаков для обнаружения объектов разного масштаба.
#    - `torchvision` предоставляет SSDlite (облегченная версия) и VGG-based SSD.
#    - Пример загрузки: `models.detection.ssd300_vgg16(pretrained=True)`
#    - Пример загрузки: `models.detection.ssdlite320_mobilenet_v3_large(pretrained=True)`

# 5. FCOS (Fully Convolutional One-Stage Object Detection):
#    - Одноэтапный детектор без "якорей" (anchor-free). Предсказывает объекты напрямую.
#    - Пример загрузки: `models.detection.fcos_resnet50_fpn(pretrained=True)`

# --------------------------------------------------

# Блок 4: Подготовка Данных для Обнаружения

# Это критически важный этап. Модели обнаружения ожидают данные в определенном формате.

# Формат Аннотаций (для `torchvision` моделей):
# - Каждому изображению должен соответствовать словарь (`target`), содержащий как минимум:
#   - `boxes`: тензор типа `torch.float32` формы `[N, 4]`, где `N` - количество объектов на изображении.
#              Каждая строка содержит `[xmin, ymin, xmax, ymax]` в абсолютных пиксельных координатах.
#   - `labels`: тензор типа `torch.int64` формы `[N]`. Каждое значение - это целочисленный индекс класса для соответствующей рамки.
#              **Важно:** Класс 0 зарезервирован для фона (`__background__`). Ваши классы должны начинаться с 1.
#              Например, если у вас 2 класса ("person", "car"), их метки будут 1 и 2.
# - Словарь `target` может содержать и другие поля, например, `masks` для Mask R-CNN или `image_id`.

# Пользовательский `Dataset`:
# - Вам почти всегда нужно будет создать свой класс, наследуемый от `torch.utils.data.Dataset`.
# - Метод `__getitem__(self, idx)` должен возвращать:
#   - Изображение (обычно как PIL Image или тензор).
#   - Словарь `target` с аннотациями в описанном выше формате.
# - Метод `__len__(self)` должен возвращать общее количество изображений в датасете.

# Пример структуры `__getitem__`:
# def __getitem__(self, idx):
#     img_path = self.image_files[idx]
#     img = Image.open(img_path).convert("RGB")
#     # Загрузка аннотаций для этого изображения (например, из XML, JSON файла)
#     annotations = self.load_annotations(idx) # Ваша функция загрузки
#     boxes = annotations['boxes'] # Получить список [[xmin, ymin, xmax, ymax], ...]
#     labels = annotations['labels'] # Получить список [label1, label2, ...]
#
#     target = {}
#     target["boxes"] = torch.as_tensor(boxes, dtype=torch.float32)
#     target["labels"] = torch.as_tensor(labels, dtype=torch.int64)
#     # target["image_id"] = torch.tensor([idx]) # Полезно для оценки
#     # target["area"] = (target["boxes"][:, 3] - target["boxes"][:, 1]) * (target["boxes"][:, 2] - target["boxes"][:, 0])
#     # target["iscrowd"] = torch.zeros((len(boxes),), dtype=torch.int64) # Обычно 0 для не-COCO датасетов
#
#     # Применение трансформаций (если есть)
#     if self.transforms is not None:
#         # Важно: Трансформации должны применяться и к изображению, и к рамкам!
#         # Библиотеки типа Albumentations хорошо подходят для этого.
#         # Стандартные torchvision transforms не всегда корректно работают с рамками.
#         img, target = self.transforms(img, target) # Пример сигнатуры трансформера
#
#     return img, target

# Трансформации и Аугментация:
# - Изменение размера, кроп, отражения и т.д. должны применяться СИНХРОННО к изображению и его рамкам.
# - Библиотека `Albumentations` очень популярна для задач обнаружения и сегментации, так как она умеет корректно трансформировать и рамки/маски.
# - Стандартные `torchvision.transforms` в основном предназначены для классификации и могут некорректно обрабатывать bounding boxes.

# `DataLoader` и `collate_fn`:
# - Поскольку каждое изображение может иметь разное количество объектов (и, следовательно, разный размер тензоров `boxes` и `labels` в `target`), стандартный `collate_fn` в `DataLoader` не сработает.
# - Нужно предоставить свою функцию `collate_fn`, которая будет принимать список кортежей `(image, target)` из датасета и правильно их батчевать. Обычно изображения собираются в тензор, а таргеты остаются списком словарей.
def collate_fn(batch):
    # batch - это список кортежей [(img1, target1), (img2, target2), ...]
    # Функция list(zip(*batch)) преобразует его в ([img1, img2, ...], [target1, target2, ...])
    return tuple(zip(*batch))

# train_dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=4, collate_fn=collate_fn)

# --------------------------------------------------

# Блок 5: Использование Предобученных Моделей и Fine-tuning

# Загрузка Предобученной Модели:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

# model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# Модификация для Своего Числа Классов:
# Предобученные модели (на COCO) имеют классификатор, рассчитанный на 91 класс (80 + фон).
# Вам нужно заменить "голову" классификатора на новую, с нужным количеством выходов.
# **Помните про класс фона!** Если у вас `N` классов, нужно `N + 1` выходов.

# num_classes = 3 # Например, 2 ваших класса + 1 фон (__background__)
#
# # Получаем количество входных признаков для классификатора
# in_features = model.roi_heads.box_predictor.cls_score.in_features
#
# # Заменяем предобученную голову на новую
# model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)

# Загрузка Модели с Другим Backbone (если нужно):
# backbone = torchvision.models.mobilenet_v2(pretrained=True).features
# backbone.out_channels = 1280 # Указать количество выходных каналов backbone
#
# # Настроить генератор якорей (RPN), если стандартные не подходят
# anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
#                                    aspect_ratios=((0.5, 1.0, 2.0),))
#
# # Настроить RoIPooler
# roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'], # Имена слоев backbone для RoIAlign
#                                                 output_size=7,
#                                                 sampling_ratio=2)
#
# # Собрать модель Faster R-CNN
# model_custom_backbone = FasterRCNN(backbone,
#                                    num_classes=num_classes,
#                                    rpn_anchor_generator=anchor_generator,
#                                    box_roi_pool=roi_pooler)


# Fine-tuning:
# - Обычно размораживают все слои или только часть (например, начиная с последних блоков ResNet).
# - Используют маленький learning rate.
# - Оптимизатор (Adam, SGD) передает параметры модели: `optimizer = optim.SGD(model.parameters(), ...)`
# - Планировщик LR часто полезен.

# --------------------------------------------------

# Блок 6: Обучение Модели Обнаружения

# Цикл Обучения:
# - Значительно отличается от классификации.
# - Модели `torchvision.models.detection` ведут себя по-разному в режимах `train()` и `eval()`.
#   - `model.train()`: Модель принимает на вход список изображений и список таргетов (`targets`). Возвращает словарь лоссов (`{'loss_classifier': ..., 'loss_box_reg': ..., 'loss_objectness': ..., 'loss_rpn_box_reg': ...}`).
#   - `model.eval()`: Модель принимает на вход список изображений. Возвращает список предсказаний (`[{'boxes': ..., 'labels': ..., 'scores': ...}, ...]`) для каждого изображения.

# Пример Цикла Обучения (Упрощенный):
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# model.to(device)
#
# # Параметры для оптимизатора
# params = [p for p in model.parameters() if p.requires_grad]
# optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
#
# num_epochs = 10
#
# for epoch in range(num_epochs):
#     model.train() # Установить режим обучения
#     epoch_loss = 0
#     for images, targets in train_dataloader: # Загрузчик с collate_fn
#         images = list(image.to(device) for image in images)
#         targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
#
#         # Прямой проход (в режиме train возвращает лоссы)
#         loss_dict = model(images, targets)
#         losses = sum(loss for loss in loss_dict.values()) # Суммируем все лоссы
#
#         # Обратный проход
#         optimizer.zero_grad()
#         losses.backward()
#         optimizer.step()
#
#         epoch_loss += losses.item()
#
#     # Обновить планировщик LR
#     if lr_scheduler is not None:
#         lr_scheduler.step()
#
#     print(f"Epoch {epoch}: Loss: {epoch_loss / len(train_dataloader)}")
#
#     # --- Фаза Валидации (Оценка) ---
#     # Для оценки обычно используют внешние инструменты (например, pycocotools)
#     # для расчета mAP на валидационном датасете.
#     # Это более сложный процесс, чем просто расчет accuracy.
#     # Здесь для простоты опустим детальную оценку mAP.
#     # model.eval()
#     # with torch.no_grad():
#     #     for images, targets in val_dataloader:
#     #         images = list(img.to(device) for img in images)
#     #         outputs = model(images) # Получаем предсказания
#     #         # ... (логика сравнения outputs с targets и расчета mAP) ...

# Оценка (Evaluation):
# - Требует сравнения предсказанных рамок (`outputs`) с истинными (`targets`) с использованием порога IoU.
# - Библиотека `pycocotools` является стандартом для оценки на датасетах формата COCO.
# - `torchvision` предоставляет примеры скриптов для обучения и оценки (`references/detection/`).

# --------------------------------------------------

# Блок 7: Инференс (Получение Предсказаний)

# - Перевести модель в режим оценки: `model.eval()`.
# - Подготовить входное изображение:
#   - Применить необходимые трансформации (Resize, ToTensor, Normalize), как при валидации.
#   - Добавить измерение батча (даже если изображение одно): `input_tensor = image_tensor.unsqueeze(0)`.
#   - Переместить тензор на нужное устройство (`.to(device)`).
# - Выполнить предсказание в блоке `with torch.no_grad():`.
# - Интерпретировать выход:
#   - Модель вернет список словарей (один словарь на изображение в батче).
#   - Каждый словарь содержит ключи `'boxes'`, `'labels'`, `'scores'`.
#   - Обычно фильтруют предсказания по порогу уверенности (`scores > threshold`).
#   - Координаты рамок (`boxes`) могут потребовать конвертации в `int` для отрисовки.

# Пример Инференса:
# img = Image.open("path/to/your/image.jpg").convert("RGB")
# transform = val_test_transforms # Используем трансформации валидации/теста
# input_tensor = transform(img).unsqueeze(0).to(device)
#
# model.eval()
# with torch.no_grad():
#     predictions = model(input_tensor)
#
# # predictions - это список с одним элементом (словарем) для нашего одного изображения
# pred_boxes = predictions[0]['boxes'].cpu().numpy()
# pred_labels = predictions[0]['labels'].cpu().numpy()
# pred_scores = predictions[0]['scores'].cpu().numpy()
#
# # Фильтрация по порогу уверенности
# confidence_threshold = 0.5
# filtered_indices = pred_scores > confidence_threshold
# filtered_boxes = pred_boxes[filtered_indices]
# filtered_labels = pred_labels[filtered_indices]
# filtered_scores = pred_scores[filtered_indices]
#
# # Дальнейшая обработка или визуализация filtered_boxes, filtered_labels, filtered_scores

# --------------------------------------------------

# Блок 8: Пример Задачи и Решения (Полный Код)

# --- Условие Задачи ---
# Задача: Обучить модель Faster R-CNN (с ResNet50-FPN backbone) обнаруживать
# два класса объектов: "Красный квадрат" (метка 1) и "Синий круг" (метка 2)
# на синтетических изображениях. Использовать предобученные веса COCO и дообучить модель.

# --- Решение (Полный Код) ---

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import time
import os
import copy

# --- 0. Настройки ---
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}")

NUM_CLASSES = 3 # 2 класса (квадрат, круг) + 1 фон
BATCH_SIZE = 2 # Маленький батч для примера
NUM_EPOCHS = 5 # Мало эпох для быстрого примера
LEARNING_RATE = 0.001
IMG_SIZE = 256 # Размер синтетических изображений

# --- 1. Создание Синтетического Датасета ---
class SyntheticShapesDataset(Dataset):
    def __init__(self, num_samples=100, img_size=256, num_classes=3, transform=None):
        self.num_samples = num_samples
        self.img_size = img_size
        self.num_classes = num_classes # Включая фон
        self.transform = transform # Пока не используем сложные трансформации
        self.images = []
        self.targets = []
        self._generate_data()

    def _generate_data(self):
        print("Generating synthetic data...")
        for idx in range(self.num_samples):
            img = Image.new('RGB', (self.img_size, self.img_size), color='white')
            draw = ImageDraw.Draw(img)
            boxes = []
            labels = []

            # Добавим 1-3 фигуры
            num_shapes = np.random.randint(1, 4)
            for _ in range(num_shapes):
                shape_type = np.random.choice(['square', 'circle'])
                size = np.random.randint(20, 60)
                x = np.random.randint(0, self.img_size - size)
                y = np.random.randint(0, self.img_size - size)
                xmin, ymin, xmax, ymax = x, y, x + size, y + size

                if shape_type == 'square':
                    color = 'red'
                    label = 1 # Метка для красного квадрата
                    draw.rectangle([xmin, ymin, xmax, ymax], fill=color, outline='black')
                else: # circle
                    color = 'blue'
                    label = 2 # Метка для синего круга
                    draw.ellipse([xmin, ymin, xmax, ymax], fill=color, outline='black')

                boxes.append([xmin, ymin, xmax, ymax])
                labels.append(label)

            self.images.append(img)
            target = {}
            target["boxes"] = torch.as_tensor(boxes, dtype=torch.float32)
            target["labels"] = torch.as_tensor(labels, dtype=torch.int64)
            target["image_id"] = torch.tensor([idx])
            # Добавим area и iscrowd для совместимости с некоторыми функциями оценки
            if boxes:
                 target["area"] = (target["boxes"][:, 3] - target["boxes"][:, 1]) * (target["boxes"][:, 2] - target["boxes"][:, 0])
            else:
                 target["area"] = torch.empty((0,), dtype=torch.float32)
            target["iscrowd"] = torch.zeros((len(boxes),), dtype=torch.int64)
            self.targets.append(target)
        print("Synthetic data generated.")

    def __getitem__(self, idx):
        img = self.images[idx]
        target = self.targets[idx]

        # Простое преобразование в тензор
        img_tensor = torchvision.transforms.functional.to_tensor(img)

        # В реальной задаче здесь были бы сложные трансформации (Albumentations)
        # if self.transform:
        #     # Применить трансформации к img и target['boxes']
        #     pass

        return img_tensor, target

    def __len__(self):
        return self.num_samples

# --- 2. Загрузчики Данных ---
def collate_fn_shapes(batch):
    return tuple(zip(*batch))

train_dataset = SyntheticShapesDataset(num_samples=80, img_size=IMG_SIZE, num_classes=NUM_CLASSES)
val_dataset = SyntheticShapesDataset(num_samples=20, img_size=IMG_SIZE, num_classes=NUM_CLASSES) # Генерируем отдельно для валидации

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                          collate_fn=collate_fn_shapes, num_workers=0) # num_workers=0 для Windows/простоты
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False,
                        collate_fn=collate_fn_shapes, num_workers=0)

# --- 3. Загрузка и Модификация Модели ---
def get_model_instance_detection(num_classes):
    # Загружаем предобученную модель Faster R-CNN
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=torchvision.models.detection.FasterRCNN_ResNet50_FPN_Weights.DEFAULT)

    # Получаем количество входных признаков для классификатора
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # Заменяем предобученную голову на новую
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model

model = get_model_instance_detection(NUM_CLASSES)
model.to(device)

# --- 4. Оптимизатор и Планировщик ---
params = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.Adam(params, lr=LEARNING_RATE) # Adam может быть проще для старта
# optimizer = torch.optim.SGD(params, lr=LEARNING_RATE, momentum=0.9, weight_decay=0.0005)
# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# --- 5. Цикл Обучения ---
print("\nStarting Training...")
training_start_time = time.time()

for epoch in range(NUM_EPOCHS):
    model.train()
    epoch_loss = 0
    batch_count = 0
    for images, targets in train_loader:
        batch_start_time = time.time()
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        epoch_loss += losses.item()
        batch_count += 1
        batch_end_time = time.time()
        # print(f"  Batch {batch_count}/{len(train_loader)}, Loss: {losses.item():.4f}, Time: {batch_end_time - batch_start_time:.2f}s")


    avg_epoch_loss = epoch_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Average Loss: {avg_epoch_loss:.4f}")

    # Простая валидация (просто прогоняем и смотрим лосс, без mAP)
    model.eval()
    val_loss = 0
    with torch.no_grad():
         for images, targets in val_loader:
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            # В режиме eval модель возвращает предсказания, но если передать таргеты,
            # она все равно может посчитать лоссы (удобно для валидации лосса)
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
            val_loss += losses.item()
    avg_val_loss = val_loss / len(val_loader)
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Validation Loss: {avg_val_loss:.4f}")


    # if lr_scheduler:
    #     lr_scheduler.step()

training_end_time = time.time()
print(f"Training finished in {training_end_time - training_start_time:.2f} seconds.")

# --- 6. Инференс и Визуализация ---
print("\nRunning Inference on a sample image...")

# Возьмем изображение из валидационного набора
img_tensor, target = val_dataset[np.random.randint(len(val_dataset))] # Случайное изображение
img_pil = torchvision.transforms.ToPILImage()(img_tensor) # Конвертируем обратно в PIL для отрисовки

model.eval()
with torch.no_grad():
    prediction = model([img_tensor.to(device)]) # Подаем как список

# Извлекаем предсказания (для первого изображения в батче)
pred_boxes = prediction[0]['boxes'].cpu().numpy()
pred_labels = prediction[0]['labels'].cpu().numpy()
pred_scores = prediction[0]['scores'].cpu().numpy()

# Истинные рамки для сравнения
true_boxes = target['boxes'].cpu().numpy()
true_labels = target['labels'].cpu().numpy()

# Функция для отрисовки
def plot_image_with_boxes(image, pred_boxes, pred_labels, pred_scores, true_boxes, true_labels, threshold=0.5):
    fig, ax = plt.subplots(1, figsize=(8, 8))
    ax.imshow(image)

    # Истинные рамки (зеленые)
    for box, label in zip(true_boxes, true_labels):
        xmin, ymin, xmax, ymax = box
        width, height = xmax - xmin, ymax - ymin
        rect = patches.Rectangle((xmin, ymin), width, height, linewidth=2, edgecolor='g', facecolor='none')
        ax.add_patch(rect)
        label_text = f"True: {label}" # 1: Square, 2: Circle
        ax.text(xmin, ymin - 5, label_text, color='green', fontsize=10, bbox=dict(facecolor='white', alpha=0.5, pad=0))


    # Предсказанные рамки (красные)
    for box, label, score in zip(pred_boxes, pred_labels, pred_scores):
        if score >= threshold:
            xmin, ymin, xmax, ymax = box
            width, height = xmax - xmin, ymax - ymin
            rect = patches.Rectangle((xmin, ymin), width, height, linewidth=2, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
            label_text = f"Pred: {label} ({score:.2f})"
            ax.text(xmin, ymax + 15, label_text, color='red', fontsize=10, bbox=dict(facecolor='white', alpha=0.5, pad=0))

    plt.axis('off')
    plt.show()

# Отрисовка результата
plot_image_with_boxes(img_pil, pred_boxes, pred_labels, pred_scores, true_boxes, true_labels, threshold=0.5)

print("Inference and visualization complete.")
# --- Конец Примера ---

# --------------------------------------------------

In [ ]:
# Блок 1: Введение в Сегментацию Изображений

# Что такое Сегментация Изображений?
# Это задача Computer Vision, цель которой - разделить изображение на несколько сегментов (областей),
# присваивая каждому пикселю изображения метку класса.
# В отличие от классификации (1 метка на изображение) или детекции (рамки вокруг объектов),
# сегментация обеспечивает понимание содержимого изображения на уровне пикселей.

# Цель Сегментации:
# Создать детальную карту изображения, где каждый пиксель отнесен к определенному классу объектов или фону.

# Типы Сегментации:
# 1. Семантическая Сегментация (Semantic Segmentation):
#    - Каждому пикселю присваивается метка класса (например, "дорога", "здание", "небо", "человек").
#    - Не различает экземпляры одного класса (все люди имеют одну метку).
#    - Этот туториал фокусируется в основном на семантической сегментации.
# 2. Экземплярная Сегментация (Instance Segmentation):
#    - Каждому пикселю присваивается метка класса И идентификатор экземпляра.
#    - Различает отдельные объекты одного класса (например, "человек 1", "человек 2").
#    - Mask R-CNN (из раздела детекции) является популярной моделью для этой задачи.
# 3. Паноптическая Сегментация (Panoptic Segmentation):
#    - Объединяет семантическую и экземплярную сегментацию.
#    - Каждому пикселю присваивается метка класса и (если пиксель принадлежит объекту-"экземпляру") идентификатор экземпляра.

# --------------------------------------------------

# Блок 2: Ключевые Концепции

# 1. Маска Сегментации (Segmentation Mask):
#    - Выход модели сегментации. Представляет собой изображение (или тензор),
#      того же размера, что и входное изображение.
#    - Каждый пиксель маски содержит целочисленное значение - ID класса, к которому относится
#      соответствующий пиксель входного изображения.
#    - Например, 0 - фон, 1 - дорога, 2 - здание, 3 - человек.
#    - Формат: Обычно тензор `[H, W]` типа `torch.int64`.

# 2. Архитектуры "Энкодер-Декодер":
#    - Многие современные модели сегментации (FCN, U-Net, DeepLab) используют эту структуру.
#    - Энкодер (Encoder): Обычно это предобученная классификационная сеть (ResNet, MobileNet),
#      которая извлекает иерархические признаки и уменьшает пространственное разрешение (downsampling).
#      Захватывает семантический контекст.
#    - Декодер (Decoder): Постепенно восстанавливает пространственное разрешение (upsampling),
#      используя признаки из энкодера (часто с помощью skip connections), чтобы создать
#      полноразмерную маску сегментации. Уточняет локализацию.

# 3. Skip Connections:
#    - Соединения, передающие признаки из слоев энкодера напрямую в соответствующие (по разрешению)
#      слои декодера.
#    - Помогают декодеру использовать как низкоуровневые (детализированные), так и высокоуровневые
#      (семантические) признаки для более точной сегментации границ объектов.
#    - Ключевая идея в архитектурах типа U-Net и FPN (используется в DeepLabV3).

# 4. Транспонированная Свертка (Transposed Convolution / Deconvolution):
#    - Слой, используемый в декодере для увеличения пространственного разрешения карт признаков (upsampling).
#    - Можно рассматривать как "обратную" свертку. Имеет обучаемые параметры.

# 5. Atrous (Dilated) Convolution (Расширенная Свертка):
#    - Свертка с "пробелами" между весами ядра. Позволяет увеличить поле обзора (receptive field)
#      слоя без увеличения количества параметров и без уменьшения пространственного разрешения.
#    - Ключевой компонент в моделях DeepLab для захвата контекста на разных масштабах.

# 6. Atrous Spatial Pyramid Pooling (ASPP):
#    - Модуль (используется в DeepLab), который применяет несколько параллельных расширенных сверток
#      с разными коэффициентами расширения (dilation rates) к одной и той же карте признаков.
#    - Позволяет эффективно захватывать информацию об объектах и контексте на разных масштабах.

# 7. Метрики Оценки:
#    - Pixel Accuracy (Пиксельная Точность): Процент пикселей, классифицированных правильно.
#      Простая, но может быть обманчива при сильном дисбалансе классов (например, много фона).
#    - Intersection over Union (IoU) / Jaccard Index:
#      - Для одного класса: IoU = TP / (TP + FP + FN)
#        (TP - True Positives, FP - False Positives, FN - False Negatives для пикселей этого класса)
#      - Площадь пересечения предсказанной маски класса и истинной маски, деленная на площадь их объединения.
#    - Mean IoU (mIoU): IoU, усредненное по всем классам (кроме фона, иногда).
#      Стандартная и наиболее важная метрика для семантической сегментации.
#    - Dice Coefficient (Коэффициент Дайса / F1-score для пикселей):
#      - Dice = 2 * TP / (2 * TP + FP + FN) = 2 * |Intersection| / (|Prediction| + |GroundTruth|)
#      - Тесно связан с IoU: Dice = 2 * IoU / (IoU + 1). Часто используется в медицинской сегментации.

# --------------------------------------------------

# Блок 3: Модели Сегментации в `torchvision.models.segmentation`

# `torchvision` предоставляет готовые реализации семантической сегментации.

# 1. FCN (Fully Convolutional Network):
#    - Одна из первых моделей, показавших эффективность глубокого обучения для сегментации.
#    - Заменяет полносвязные слои в классификационных сетях на сверточные, позволяя обрабатывать изображения произвольного размера.
#    - Использует skip connections для комбинирования карт признаков разного разрешения.
#    - Пример загрузки: `models.segmentation.fcn_resnet50(pretrained=True)` или `fcn_resnet101`.

# 2. DeepLabV3:
#    - State-of-the-art модель, использующая расширенные свертки (atrous convolutions) и ASPP.
#    - Эффективно работает с объектами разного масштаба.
#    - Обычно использует ResNet или MobileNet как backbone.
#    - Пример загрузки: `models.segmentation.deeplabv3_resnet50(pretrained=True)`, `deeplabv3_resnet101`, `deeplabv3_mobilenet_v3_large`.

# 3. LR-ASPP (Lite R-ASPP):
#    - Облегченная версия DeepLab, использующая MobileNetV3 в качестве backbone.
#    - Предназначена для мобильных устройств с ограниченными ресурсами.
#    - Пример загрузки: `models.segmentation.lraspp_mobilenet_v3_large(pretrained=True)`.

# Предобученные Веса:
# - Модели обычно предобучены на подмножестве COCO 2017, включающем 20 классов объектов (+ 1 фон), которые также есть в Pascal VOC.
# - Классы: __background__, aeroplane, bicycle, bird, boat, bottle, bus, car, cat, chair, cow, diningtable, dog, horse, motorbike, person, pottedplant, sheep, sofa, train, tvmonitor.

# --------------------------------------------------

# Блок 4: Подготовка Данных для Сегментации

# Формат Аннотаций (Масок):
# - Каждому входному изображению `[3, H, W]` должна соответствовать маска сегментации `[H, W]`.
# - Маска должна быть тензором типа `torch.int64`.
# - Значения пикселей в маске - это ID классов: 0 для фона, 1 для класса 1, 2 для класса 2 и т.д.
# - Количество классов `num_classes` должно соответствовать максимальному ID класса + 1.

# Пользовательский `Dataset`:
# - Наследуемся от `torch.utils.data.Dataset`.
# - `__getitem__(self, idx)` возвращает:
#   - Изображение (PIL Image или тензор).
#   - Маску сегментации (PIL Image в режиме 'L' или 'P', или тензор `[H, W]`).
# - `__len__(self)` возвращает количество изображений.

# Пример структуры `__getitem__`:
# def __getitem__(self, idx):
#     img_path = self.image_files[idx]
#     mask_path = self.mask_files[idx]
#
#     img = Image.open(img_path).convert("RGB")
#     # Маски часто сохраняют как grayscale изображения, где значение пикселя = ID класса
#     mask = Image.open(mask_path) # Не конвертировать в RGB! Оставить как есть ('L' или 'P')
#
#     # Применение трансформаций СИНХРОННО к изображению и маске
#     if self.transforms is not None:
#         # Используйте библиотеки типа Albumentations или напишите свои трансформеры
#         # Важно: интерполяция для маски должна быть Nearest Neighbor, чтобы не создавать новые ID классов
#         img, mask = self.transforms(img, mask) # Пример сигнатуры
#
#     # Конвертация маски в тензор int64 ПОСЛЕ трансформаций (если они возвращают PIL)
#     # mask_tensor = torch.from_numpy(np.array(mask)).long()
#
#     return img, mask # или img_tensor, mask_tensor

# Трансформации и Аугментация (`Albumentations`):
# - Это КРИТИЧЕСКИ важно. Геометрические аугментации (повороты, масштабирование, кропы, флипы)
#   должны применяться абсолютно одинаково к изображению и маске.
# - `Albumentations` - лучшая библиотека для этого.
#   ```python
#   import albumentations as A
#   from albumentations.pytorch import ToTensorV2
#
#   train_transform = A.Compose([
#       A.Resize(height=256, width=256),
#       A.HorizontalFlip(p=0.5),
#       A.Rotate(limit=35, p=0.3),
#       A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
#       ToTensorV2(), # Конвертирует и img, и mask в тензоры
#   ])
#
#   # В __getitem__:
#   # augmented = train_transform(image=np.array(img), mask=np.array(mask))
#   # img_tensor = augmented['image']
#   # mask_tensor = augmented['mask'].long() # Убедиться, что тип long (int64)
#   # return img_tensor, mask_tensor
#   ```
# - Для валидации/теста используйте только необходимые трансформации (Resize, Normalize, ToTensorV2).

# `DataLoader`:
# - Стандартный `collate_fn` обычно работает, так как изображения и маски после трансформаций
#   имеют одинаковый размер и могут быть собраны в батчи `[N, C, H, W]` и `[N, H, W]`.
# train_loader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=4) # collate_fn не нужен

# --------------------------------------------------

# Блок 5: Использование Предобученных Моделей и Fine-tuning

# Загрузка Предобученной Модели:
import torchvision.models as models

# model = models.segmentation.deeplabv3_resnet50(pretrained=True)
# model = models.segmentation.fcn_resnet50(pretrained=True)

# Модификация для Своего Числа Классов:
# Предобученные модели (на COCO/VOC) имеют классификатор на 21 класс (20 + фон).
# Нужно заменить последний слой(и) классификатора.

# num_classes = 5 # Например, 4 ваших класса + 1 фон

# Для DeepLabV3 (имеет основной и вспомогательный классификатор)
# model = models.segmentation.deeplabv3_resnet50(weights=models.segmentation.DeepLabV3_ResNet50_Weights.DEFAULT)
# # Заменяем основной классификатор
# model.classifier[4] = nn.Conv2d(256, num_classes, kernel_size=(1, 1), stride=(1, 1))
# # Заменяем вспомогательный классификатор (если он есть и используется)
# model.aux_classifier[4] = nn.Conv2d(256, num_classes, kernel_size=(1, 1), stride=(1, 1))

# Для FCN (имеет основной и вспомогательный классификатор)
# model = models.segmentation.fcn_resnet50(weights=models.segmentation.FCN_ResNet50_Weights.DEFAULT)
# model.classifier[4] = nn.Conv2d(512, num_classes, kernel_size=(1, 1), stride=(1, 1))
# model.aux_classifier[4] = nn.Conv2d(256, num_classes, kernel_size=(1, 1), stride=(1, 1))


# Fine-tuning:
# - Аналогично классификации/детекции.
# - Можно заморозить backbone и обучать только классификаторы (Feature Extraction).
# - Можно разморозить все или часть слоев и обучать с маленьким LR (Fine-tuning).
# - Передать параметры в оптимизатор: `optimizer = optim.Adam(model.parameters(), lr=...)`

# --------------------------------------------------

# Блок 6: Обучение Модели Сегментации

# Цикл Обучения:
# - Похож на классификацию, но работает с пиксельными предсказаниями и масками.
# - Модели `torchvision.models.segmentation` возвращают словарь OrderedDict.
#   - Ключ `'out'`: Основной выход модели, тензор с логитами формы `[N, C, H, W]`, где `C` - количество классов.
#   - Ключ `'aux'` (если есть): Выход вспомогательного классификатора, той же формы.

# Функция Потерь (Loss Function):
# - `nn.CrossEntropyLoss` - стандартный выбор.
# - Она ожидает:
#   - Input (логиты): `[N, C, H, W]`
#   - Target (маска с ID классов): `[N, H, W]` типа `torch.int64`.
# - Автоматически применяет Softmax к логитам и вычисляет NLLLoss.
# - Можно задать `weight` для классов (если есть дисбаланс) или `ignore_index` (если нужно игнорировать пиксели с определенным ID в маске).

# Пример Цикла Обучения (Упрощенный):
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# model.to(device)
#
# params = [p for p in model.parameters() if p.requires_grad]
# optimizer = torch.optim.Adam(params, lr=0.001)
# criterion = nn.CrossEntropyLoss() # Веса или ignore_index можно добавить сюда
#
# num_epochs = 10
#
# for epoch in range(num_epochs):
#     model.train()
#     running_loss = 0.0
#     for images, masks in train_loader:
#         images = images.to(device)
#         masks = masks.to(device) # Ожидаемый тип: torch.int64
#
#         optimizer.zero_grad()
#
#         # Прямой проход
#         outputs = model(images) # Получаем словарь {'out': ..., 'aux': ...}
#
#         # Расчет лосса
#         loss = criterion(outputs['out'], masks)
#         # Если используется вспомогательный лосс:
#         if 'aux' in outputs:
#             loss_aux = criterion(outputs['aux'], masks)
#             loss = loss + 0.4 * loss_aux # Вес для aux_loss (часто 0.4 или 0.5)
#
#         # Обратный проход
#         loss.backward()
#         optimizer.step()
#
#         running_loss += loss.item()
#
#     epoch_loss = running_loss / len(train_loader)
#     print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}")
#
#     # --- Фаза Валидации (Оценка mIoU) ---
#     model.eval()
#     # ... (Код для расчета Pixel Accuracy и mIoU на val_loader) ...
#     # Расчет mIoU требует построения confusion matrix по всем пикселям валидационного сета.

# Оценка (mIoU):
# - Требует итерации по валидационному сету, получения предсказанных масок (`argmax` по выходу)
#   и сравнения их с истинными масками для построения суммарной матрицы ошибок (confusion matrix)
#   размера `num_classes x num_classes`.
# - Затем из матрицы ошибок вычисляется IoU для каждого класса и усредняется.

# --------------------------------------------------

# Блок 7: Инференс (Получение Предсказаний)

# - Перевести модель в режим оценки: `model.eval()`.
# - Подготовить входное изображение (трансформации как для валидации, unsqueeze(0), to(device)).
# - Выполнить предсказание `with torch.no_grad(): outputs = model(input_tensor)`.
# - Получить тензор логитов из словаря: `logits = outputs['out']`. Форма `[1, C, H, W]`.
# - Получить предсказанную маску с ID классов: `pred_mask = torch.argmax(logits, dim=1).squeeze(0).cpu().numpy()`. Форма `[H, W]`.
# - (Опционально) Применить `softmax` к `logits` для получения вероятностей для каждого пикселя/класса.
# - (Опционально) Если входное изображение изменяло размер, нужно изменить размер `pred_mask` обратно к исходному размеру изображения (используя интерполяцию `Nearest Neighbor`).
# - Визуализировать `pred_mask`, например, раскрасив пиксели в соответствии с предсказанными ID классов.

# --------------------------------------------------

# Блок 8: Пример Задачи и Решения (Полный Код)

# --- Условие Задачи ---
# Задача: Обучить модель DeepLabV3 (с ResNet50 backbone) для семантической сегментации
# двух классов объектов: "Красный квадрат" (метка 1) и "Синий круг" (метка 2)
# на синтетических изображениях. Фон имеет метку 0. Использовать предобученные веса и дообучить модель.

# --- Решение (Полный Код) ---

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.models.segmentation import deeplabv3_resnet50, DeepLabV3_ResNet50_Weights
import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import time
import os
import copy
import albumentations as A
from albumentations.pytorch import ToTensorV2

# --- 0. Настройки ---
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}")

NUM_CLASSES = 3 # 1: Квадрат, 2: Круг, 0: Фон
BATCH_SIZE = 4 # Увеличим немного батч
NUM_EPOCHS = 5 # Мало эпох для быстрого примера
LEARNING_RATE = 0.001
IMG_SIZE = 224 # DeepLab обычно работает с этим размером или больше
MODEL_SAVE_PATH = "deeplabv3_shapes_best.pth"

# --- 1. Создание Синтетического Датасета с Масками ---
class SyntheticShapesMaskDataset(Dataset):
    def __init__(self, num_samples=100, img_size=224, transform=None):
        self.num_samples = num_samples
        self.img_size = img_size
        self.transform = transform
        self.images = []
        self.masks = []
        self._generate_data()

    def _generate_data(self):
        print("Generating synthetic data with masks...")
        for idx in range(self.num_samples):
            img = Image.new('RGB', (self.img_size, self.img_size), color='white')
            mask = Image.new('L', (self.img_size, self.img_size), color=0) # Маска фона (класс 0)
            draw_img = ImageDraw.Draw(img)
            draw_mask = ImageDraw.Draw(mask)

            num_shapes = np.random.randint(1, 4)
            for _ in range(num_shapes):
                shape_type = np.random.choice(['square', 'circle'])
                size = np.random.randint(20, 60)
                x = np.random.randint(0, self.img_size - size)
                y = np.random.randint(0, self.img_size - size)
                xmin, ymin, xmax, ymax = x, y, x + size, y + size

                if shape_type == 'square':
                    img_color = 'red'
                    mask_value = 1 # Метка для квадрата
                    draw_img.rectangle([xmin, ymin, xmax, ymax], fill=img_color, outline='black')
                    draw_mask.rectangle([xmin, ymin, xmax, ymax], fill=mask_value) # Заливаем маску ID класса
                else: # circle
                    img_color = 'blue'
                    mask_value = 2 # Метка для круга
                    draw_img.ellipse([xmin, ymin, xmax, ymax], fill=img_color, outline='black')
                    draw_mask.ellipse([xmin, ymin, xmax, ymax], fill=mask_value) # Заливаем маску ID класса

            self.images.append(img)
            self.masks.append(mask)
        print("Synthetic data generated.")

    def __getitem__(self, idx):
        img = self.images[idx]
        mask = self.masks[idx]

        # Конвертируем в numpy перед Albumentations
        img_np = np.array(img)
        mask_np = np.array(mask)

        if self.transform:
            augmented = self.transform(image=img_np, mask=mask_np)
            img_tensor = augmented['image']
            mask_tensor = augmented['mask'].long() # Albumentations ToTensorV2 не делает long
        else:
            # Если нет трансформаций, просто конвертируем
            img_tensor = torchvision.transforms.functional.to_tensor(img)
            mask_tensor = torch.from_numpy(mask_np).long()

        return img_tensor, mask_tensor

    def __len__(self):
        return self.num_samples

# --- 2. Трансформации и Загрузчики Данных ---
# Используем Albumentations
train_transform = A.Compose([
    A.Resize(height=IMG_SIZE, width=IMG_SIZE), # Убедимся, что размер правильный
    A.HorizontalFlip(p=0.5),
    # A.Rotate(limit=20, p=0.3), # Можно добавить еще аугментаций
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)), # ImageNet нормы
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Resize(height=IMG_SIZE, width=IMG_SIZE),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

train_dataset = SyntheticShapesMaskDataset(num_samples=100, img_size=IMG_SIZE, transform=train_transform)
val_dataset = SyntheticShapesMaskDataset(num_samples=20, img_size=IMG_SIZE, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

# --- 3. Загрузка и Модификация Модели ---
def get_model_instance_segmentation(num_classes):
    # Загружаем предобученную модель DeepLabV3
    model = deeplabv3_resnet50(weights=DeepLabV3_ResNet50_Weights.DEFAULT)

    # Заменяем основной классификатор
    # У DeepLabV3 классификатор находится в model.classifier[4]
    model.classifier[4] = nn.Conv2d(256, num_classes, kernel_size=(1, 1), stride=(1, 1))
    # Заменяем вспомогательный классификатор (если он есть)
    # У DeepLabV3 с ResNet он в model.aux_classifier[4]
    model.aux_classifier[4] = nn.Conv2d(256, num_classes, kernel_size=(1, 1), stride=(1, 1))

    return model

model = get_model_instance_segmentation(NUM_CLASSES)
model.to(device)

# --- 4. Оптимизатор и Функция Потерь ---
params = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.Adam(params, lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss() # Стандартный лосс для семантической сегментации

# --- 5. Цикл Обучения ---
print("\nStarting Training...")
training_start_time = time.time()
best_val_loss = float('inf')

for epoch in range(NUM_EPOCHS):
    model.train()
    epoch_loss = 0
    for images, masks in train_loader:
        images = images.to(device)
        masks = masks.to(device) # Тип должен быть Long (int64)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs['out'], masks)
        if 'aux' in outputs:
            loss_aux = criterion(outputs['aux'], masks)
            loss = loss + 0.4 * loss_aux

        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    avg_epoch_loss = epoch_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Train Loss: {avg_epoch_loss:.4f}")

    # Валидация
    model.eval()
    val_loss = 0
    # Здесь можно добавить расчет mIoU, но для простоты считаем только лосс
    with torch.no_grad():
        for images, masks in val_loader:
            images = images.to(device)
            masks = masks.to(device)
            outputs = model(images)
            loss = criterion(outputs['out'], masks) # Считаем лосс только по основному выходу на валидации
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Validation Loss: {avg_val_loss:.4f}")

    # Сохранение лучшей модели по валидационному лоссу
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
        print(f"Saved best model to {MODEL_SAVE_PATH}")

training_end_time = time.time()
print(f"Training finished in {training_end_time - training_start_time:.2f} seconds.")

# --- 6. Инференс и Визуализация ---
print("\nLoading best model for inference...")
# Загружаем лучшую модель
model = get_model_instance_segmentation(NUM_CLASSES) # Создаем инстанс
model.load_state_dict(torch.load(MODEL_SAVE_PATH))
model.to(device)
model.eval()

print("Running Inference on a sample image...")
# Возьмем изображение из валидационного набора
img_tensor, true_mask_tensor = val_dataset[np.random.randint(len(val_dataset))]

# Денормализация и конвертация для визуализации
inv_normalize = transforms.Normalize(
    mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
    std=[1/0.229, 1/0.224, 1/0.225]
)
img_vis = inv_normalize(img_tensor).permute(1, 2, 0).cpu().numpy() # CHW -> HWC
img_vis = (img_vis * 255).astype(np.uint8)
true_mask_vis = true_mask_tensor.cpu().numpy()

# Получение предсказания
with torch.no_grad():
    input_batch = img_tensor.unsqueeze(0).to(device)
    output = model(input_batch)['out'] # Берем основной выход
    pred_mask = torch.argmax(output, dim=1).squeeze(0).cpu().numpy() # NCHW -> HW

# Функция для отрисовки масок с цветами
def decode_segmap(mask, num_classes):
    # Простая палитра: 0-черный(фон), 1-красный(квадрат), 2-синий(круг)
    palette = np.array([[0, 0, 0], [255, 0, 0], [0, 0, 255]], dtype=np.uint8)
    rgb_mask = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
    for label in range(num_classes):
        rgb_mask[mask == label] = palette[label]
    return rgb_mask

# Визуализация
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].imshow(img_vis)
ax[0].set_title('Input Image')
ax[0].axis('off')

ax[1].imshow(decode_segmap(true_mask_vis, NUM_CLASSES))
ax[1].set_title('Ground Truth Mask')
ax[1].axis('off')

ax[2].imshow(decode_segmap(pred_mask, NUM_CLASSES))
ax[2].set_title('Predicted Mask')
ax[2].axis('off')

plt.tight_layout()
plt.show()

print("Inference and visualization complete.")
# --- Конец Примера ---

# --------------------------------------------------

In [ ]:
# Блок 1: Введение в Фильтрацию Изображений

# Что такое Фильтрация Изображений?
# Это процесс модификации или улучшения изображения путем применения математических операций
# к каждому пикселю и его соседям. Эти операции обычно выполняются с помощью "ядра" или "фильтра".
# Фильтрация является фундаментальной операцией во многих задачах Computer Vision и обработки изображений.

# Цель Фильтрации:
# - Снижение шума (Noise Reduction).
# - Увеличение резкости (Sharpening) и выделение деталей.
# - Размытие (Blurring) для сглаживания или как предварительный этап для других операций.
# - Обнаружение краев (Edge Detection).
# - Извлечение определенных признаков.

# Как это работает (Концепция Конволюции):
# - Маленькая матрица (ядро/фильтр) скользит по всему изображению.
# - В каждой позиции вычисляется взвешенная сумма значений пикселей, попадающих под ядро.
# - Веса задаются значениями в ядре.
# - Результат этой суммы становится новым значением центрального пикселя (под якорем ядра) в выходном изображении.
# - Этот процесс называется конволюцией (или корреляцией, в зависимости от реализации).

# --------------------------------------------------

# Блок 2: Ядра (Фильтры) и Конволюция

# Ядро (Kernel / Filter / Mask):
# - Небольшая 2D матрица чисел.
# - Размер ядра (например, 3x3, 5x5) определяет окрестность пикселя, которая учитывается при фильтрации.
# - Значения в ядре определяют характер фильтрации (размытие, резкость и т.д.).
# - Якорь (Anchor Point): Центральный элемент ядра, который совмещается с текущим пикселем изображения.

# Пример ядра 3x3 (усредняющий фильтр):
# kernel_avg = [[1/9, 1/9, 1/9],
#               [1/9, 1/9, 1/9],
#               [1/9, 1/9, 1/9]]

# Пример ядра 3x3 (увеличение резкости):
# kernel_sharpen = [[ 0, -1,  0],
#                   [-1,  5, -1],
#                   [ 0, -1,  0]]

# Обработка Границ (Padding):
# - Когда ядро находится у края изображения, часть его выходит за пределы.
# - Способы обработки:
#   - Игнорировать пиксели у края (выходное изображение будет меньше).
#   - Дополнить изображение (Padding):
#     - Нулевое дополнение (Zero Padding): Добавить пиксели со значением 0.
#     - Дополнение репликацией (Replicate): Повторить значения крайних пикселей.
#     - Дополнение отражением (Reflect): Отразить пиксели относительно границы.
# - Библиотеки (как OpenCV) обычно предоставляют опции для выбора способа обработки границ.

# --------------------------------------------------

# Блок 3: Линейные Фильтры

# Линейные фильтры выполняют взвешенное суммирование пикселей в окрестности.

# 1. Усредняющий Фильтр (Averaging / Box Filter):
#    - Ядро: Все элементы равны `1 / (ширина * высота ядра)`.
#    - Принцип: Заменяет пиксель средним значением его соседей (включая себя).
#    - Цель: Простое размытие, снижение шума. Может сильно размывать края.
#    - Реализация (OpenCV): `cv2.blur(image, (kernel_width, kernel_height))`
import cv2
import numpy as np

def apply_average_blur(image, kernel_size=(5, 5)):
    # kernel_size должен быть кортежем с нечетными числами, например (3, 3), (5, 5)
    blurred_image = cv2.blur(image, kernel_size)
    return blurred_image

# 2. Гауссовский Фильтр (Gaussian Filter):
#    - Ядро: Значения определяются 2D функцией Гаусса. Центральные пиксели имеют больший вес,
#      вес уменьшается по мере удаления от центра.
#    - Принцип: Взвешенное усреднение, где вес соседей зависит от их удаленности от центра по Гауссу.
#    - Цель: Более гладкое и естественное размытие по сравнению с усредняющим фильтром.
#      Очень эффективен для удаления Гауссовского шума. Часто используется как препроцессинг.
#    - Параметры:
#      - `ksize`: Размер ядра (ширина, высота), должны быть нечетными положительными числами.
#      - `sigmaX`: Стандартное отклонение Гаусса по оси X. Контролирует степень размытия по горизонтали.
#      - `sigmaY` (опционально): Стандартное отклонение по оси Y. Если 0, вычисляется из `sigmaX`. Если оба 0, вычисляются из `ksize`.
#    - Реализация (OpenCV): `cv2.GaussianBlur(image, ksize, sigmaX, sigmaY=...)`
def apply_gaussian_blur(image, kernel_size=(5, 5), sigmaX=0):
    # kernel_size должен быть кортежем с нечетными положительными числами
    # sigmaX=0 означает, что сигма будет вычислена из размера ядра
    blurred_image = cv2.GaussianBlur(image, kernel_size, sigmaX)
    return blurred_image

# 3. Фильтр Резкости (Sharpening Filter):
#    - Ядро: Обычно имеет положительное значение в центре и отрицательные значения вокруг,
#      сумма элементов ядра часто равна 1 (чтобы сохранить общую яркость).
#      Пример: [[0, -1, 0], [-1, 5, -1], [0, -1, 0]]
#    - Принцип: Увеличивает разницу между пикселем и его соседями, подчеркивая детали и края.
#    - Цель: Сделать изображение более четким. Может усилить существующий шум.
#    - Реализация (OpenCV): Используется общая функция конволюции `cv2.filter2D`.
def apply_sharpening(image):
    # Пример ядра для увеличения резкости
    kernel = np.array([[ 0, -1,  0],
                       [-1,  5, -1],
                       [ 0, -1,  0]])
    # Применяем ядро с помощью cv2.filter2D
    # -1 означает, что глубина выходного изображения будет такой же, как у входного
    sharpened_image = cv2.filter2D(image, -1, kernel)
    return sharpened_image

# --------------------------------------------------

# Блок 4: Нелинейные Фильтры

# Нелинейные фильтры выполняют операции, не являющиеся простым взвешенным суммированием (например, сортировка, выбор медианы/минимума/максимума).

# 1. Медианный Фильтр (Median Filter):
#    - Принцип: Заменяет значение центрального пикселя медианным значением всех пикселей в окрестности ядра.
#      (Медиана - значение, которое находится в середине отсортированного списка пикселей).
#    - Цель: Очень эффективен для удаления импульсного шума ("соль и перец").
#      Хорошо сохраняет края по сравнению с линейными фильтрами размытия.
#    - Параметры:
#      - `ksize`: Размер апертуры (сторона квадратного ядра), должно быть нечетным целым числом больше 1 (например, 3, 5).
#    - Реализация (OpenCV): `cv2.medianBlur(image, ksize)`
def apply_median_blur(image, ksize=5):
    # ksize должно быть нечетным целым > 1
    filtered_image = cv2.medianBlur(image, ksize)
    return filtered_image

# 2. Билатеральный Фильтр (Bilateral Filter):
#    - Принцип: Учитывает не только пространственную близость пикселей (как Гауссовский фильтр),
#      но и их схожесть по интенсивности/цвету. Пиксели, которые находятся близко пространственно,
#      но сильно отличаются по значению, будут иметь меньший вес.
#    - Цель: Сглаживание шума при сохранении резких краев.
#    - Параметры:
#      - `d`: Диаметр окрестности пикселя. -1 означает, что он будет вычислен из `sigmaSpace`.
#      - `sigmaColor`: Сигма в цветовом пространстве. Большее значение означает, что более удаленные по цвету пиксели будут влиять друг на друга.
#      - `sigmaSpace`: Сигма в координатном пространстве (аналогично Гауссову размытию). Большее значение означает, что более удаленные пиксели будут влиять друг на друга.
#    - Реализация (OpenCV): `cv2.bilateralFilter(image, d, sigmaColor, sigmaSpace)`
def apply_bilateral_filter(image, d=9, sigmaColor=75, sigmaSpace=75):
    # d: Диаметр окрестности.
    # sigmaColor: Фильтр сигма в цветовом пространстве.
    # sigmaSpace: Фильтр сигма в координатном пространстве.
    filtered_image = cv2.bilateralFilter(image, d, sigmaColor, sigmaSpace)
    return filtered_image

# --------------------------------------------------

# Блок 5: Библиотеки для Фильтрации

# 1. OpenCV (`cv2`):
#    - Де-факто стандарт для задач Computer Vision в Python.
#    - Предоставляет высокооптимизированные реализации большинства фильтров.
#    - Рекомендуется для большинства задач фильтрации.

# 2. SciPy (`scipy.ndimage`):
#    - Модуль `scipy.ndimage.filters` содержит функции для Гауссовой, медианной, усредняющей и общей конволюции (`convolve`).
#    - Хорошая альтернатива, особенно если вы уже работаете в экосистеме SciPy/NumPy.
#    - `from scipy.ndimage import gaussian_filter, median_filter`

# 3. Pillow (`PIL.ImageFilter`):
#    - Предоставляет базовые фильтры (BLUR, CONTOUR, DETAIL, EDGE_ENHANCE, SHARPEN, SMOOTH, GaussianBlur, MedianFilter, UnsharpMask).
#    - Менее гибкая и производительная, чем OpenCV или SciPy, для сложных задач.
#    - `from PIL import Image, ImageFilter`
#    - `im_blurred = im.filter(ImageFilter.GaussianBlur(radius=2))`

# 4. PyTorch (`kornia` или `torch.nn.functional.conv2d`):
#    - Можно реализовать линейные фильтры с помощью `F.conv2d`, определив ядро как веса свертки.
#    - Библиотека `Kornia` предоставляет GPU-ускоренные функции для многих операций CV, включая фильтрацию (GaussianBlur, MedianBlur и т.д.), совместимые с тензорами PyTorch.
#    - Обычно используется, когда фильтрация является частью большей модели глубокого обучения, выполняемой на GPU.

# --------------------------------------------------

# Блок 6: Пример Задачи и Решения (Используя OpenCV)

# --- Условие Задачи ---
# Задача: Загрузить изображение, добавить к нему искусственный шум "соль и перец".
# Затем применить Гауссовский и Медианный фильтры для удаления этого шума.
# Сравнить результаты фильтрации с исходным и зашумленным изображениями.

# --- Решение (Полный Код) ---

import cv2
import numpy as np
import matplotlib.pyplot as plt
import os # Для проверки существования файла

# --- Функция для добавления шума "соль и перец" ---
def add_salt_pepper_noise(image, salt_prob=0.02, pepper_prob=0.02):
    noisy_image = np.copy(image)
    total_pixels = image.size
    # Добавляем "соль" (белые пиксели)
    num_salt = np.ceil(salt_prob * total_pixels)
    coords = [np.random.randint(0, i - 1, int(num_salt)) for i in image.shape[:2]]
    noisy_image[coords[0], coords[1]] = 255 # Для grayscale/color

    # Добавляем "перец" (черные пиксели)
    num_pepper = np.ceil(pepper_prob * total_pixels)
    coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in image.shape[:2]]
    noisy_image[coords[0], coords[1]] = 0

    return noisy_image

# --- 1. Загрузка Изображения ---
# Укажите путь к вашему изображению
image_path = 'path/to/your/image.jpg' # ЗАМЕНИТЕ НА ВАШ ПУТЬ

# Проверка существования файла
if not os.path.exists(image_path):
    print(f"Ошибка: Файл не найден по пути: {image_path}")
    print("Пожалуйста, замените 'path/to/your/image.jpg' на корректный путь к вашему изображению.")
    # Создадим простое серое изображение как заглушку, если файл не найден
    original_image = np.full((200, 300, 3), 128, dtype=np.uint8) # Серое изображение 200x300
    print("Используется серое изображение-заглушка.")
else:
    original_image = cv2.imread(image_path)
    # Проверка успешности загрузки
    if original_image is None:
        print(f"Ошибка: Не удалось загрузить изображение по пути: {image_path}")
        exit()
    print(f"Изображение '{image_path}' успешно загружено.")

# Конвертируем в RGB для Matplotlib (OpenCV читает как BGR)
original_image_rgb = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

# --- 2. Добавление Шума ---
noisy_image = add_salt_pepper_noise(original_image_rgb)
print("Шум 'соль и перец' добавлен.")

# --- 3. Применение Гауссовского Фильтра ---
# Параметры: размер ядра (должен быть нечетным), sigmaX
gaussian_blurred = cv2.GaussianBlur(noisy_image, (5, 5), 0)
print("Гауссовский фильтр применен.")

# --- 4. Применение Медианного Фильтра ---
# Параметры: размер апертуры (нечетное число > 1)
median_filtered = cv2.medianBlur(noisy_image, 5)
print("Медианный фильтр применен.")

# --- 5. Отображение Результатов ---
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.imshow(original_image_rgb)
plt.title('Original Image')
plt.axis('off')

plt.subplot(2, 2, 2)
plt.imshow(noisy_image)
plt.title('Noisy Image (Salt & Pepper)')
plt.axis('off')

plt.subplot(2, 2, 3)
plt.imshow(gaussian_blurred)
plt.title('Gaussian Filtered')
plt.axis('off')

plt.subplot(2, 2, 4)
plt.imshow(median_filtered)
plt.title('Median Filtered')
plt.axis('off')

plt.tight_layout()
plt.show()

print("Отображение результатов завершено.")
# Наблюдение: Медианный фильтр обычно гораздо лучше справляется с шумом "соль и перец",
# в то время как Гауссовский фильтр просто размывает шумные пиксели вместе с остальным изображением.

# --- Конец Примера ---

# --------------------------------------------------

In [ ]:
# Блок 1: Шум в Изображениях - Что это и Зачем Бороться?

# Что такое Шум?
# Шум в изображениях - это случайные, нежелательные вариации яркости или цветовой информации.
# Он искажает исходное визуальное содержимое изображения.
# Источники шума:
# - Сенсор камеры (тепловой шум, шум считывания при слабом освещении).
# - Ошибки передачи данных.
# - Сжатие изображения с потерями.
# - Сканирование фотографий.

# Почему Шум - это Проблема?
# - Ухудшает визуальное качество изображения для человека.
# - Значительно затрудняет работу алгоритмов Computer Vision:
#   - Снижает точность извлечения признаков (SIFT, ORB и т.д.).
#   - Ухудшает качество сегментации и обнаружения объектов.
#   - Может привести к неверной классификации.

# Цель Борьбы с Шумом (Шумоподавление / Denoising):
# - Улучшить визуальное восприятие изображения.
# - Повысить производительность и точность последующих этапов анализа изображений.
# - Восстановить исходное изображение, максимально удалив шум и сохранив важные детали (края, текстуры).

# --------------------------------------------------

# Блок 2: Распространенные Типы Шума и Их Генерация

# Понимание типа шума важно для выбора правильного метода фильтрации.

# 1. Гауссовский Шум (Gaussian Noise):
#    - Характер: Аддитивный шум, значения которого распределены по нормальному (Гауссову) закону с нулевым средним.
#      Затрагивает каждый пиксель изображения. `I_noisy = I_original + Noise_gaussian`
#    - Причины: Часто возникает из-за теплового шума сенсора и шума считывания.
#    - Генерация (NumPy/OpenCV):
import numpy as np
import cv2

def add_gaussian_noise(image, mean=0, sigma=25):
    # Убедимся, что изображение в формате float для добавления шума
    if image.dtype != np.float64:
        image_float = image.astype(np.float64)
    else:
        image_float = image.copy()

    # Генерируем гауссовский шум
    noise = np.random.normal(mean, sigma, image.shape)
    noisy_image_float = image_float + noise

    # Обрезаем значения, чтобы они остались в допустимом диапазоне [0, 255]
    noisy_image_clipped = np.clip(noisy_image_float, 0, 255)

    # Возвращаем к исходному типу данных (обычно uint8)
    noisy_image = noisy_image_clipped.astype(image.dtype)
    return noisy_image

# 2. Шум "Соль и Перец" (Salt-and-Pepper Noise):
#    - Характер: Импульсный шум. Случайные пиксели заменяются на минимальное (0, "перец")
#      или максимальное (255, "соль") значение интенсивности.
#    - Причины: Ошибки передачи данных, дефекты ячеек памяти или сенсора.
#    - Генерация (NumPy/OpenCV):
def add_salt_pepper_noise(image, salt_prob=0.02, pepper_prob=0.02):
    noisy_image = np.copy(image)
    total_pixels = image.size
    num_salt = int(salt_prob * total_pixels)
    num_pepper = int(pepper_prob * total_pixels)

    # Добавляем "соль"
    salt_coords = [np.random.randint(0, i - 1, num_salt) for i in image.shape[:2]]
    if len(image.shape) == 3: # Цветное изображение
        salt_channels = np.random.randint(0, image.shape[2], num_salt)
        noisy_image[salt_coords[0], salt_coords[1], salt_channels] = 255
        # Можно установить все каналы в 255: noisy_image[salt_coords[0], salt_coords[1], :] = 255
    else: # Grayscale
        noisy_image[salt_coords[0], salt_coords[1]] = 255

    # Добавляем "перец"
    pepper_coords = [np.random.randint(0, i - 1, num_pepper) for i in image.shape[:2]]
    if len(image.shape) == 3: # Цветное изображение
        pepper_channels = np.random.randint(0, image.shape[2], num_pepper)
        noisy_image[pepper_coords[0], pepper_coords[1], pepper_channels] = 0
        # Можно установить все каналы в 0: noisy_image[pepper_coords[0], pepper_coords[1], :] = 0
    else: # Grayscale
        noisy_image[pepper_coords[0], pepper_coords[1]] = 0

    return noisy_image

# 3. Пятнистый Шум (Speckle Noise):
#    - Характер: Мультипликативный шум. `I_noisy = I_original * (1 + Noise)` или `I_noisy = I_original + I_original * Noise`.
#      Интенсивность шума пропорциональна интенсивности пикселя.
#    - Причины: Часто встречается в когерентных системах визуализации (радары (SAR), УЗИ, лазеры).
#    - Генерация (NumPy/OpenCV):
def add_speckle_noise(image, sigma=0.1):
     # Убедимся, что изображение в формате float
    if image.dtype != np.float64:
        image_float = image.astype(np.float64) / 255.0 # Нормализуем в [0, 1]
    else:
        image_float = image.copy()

    # Генерируем гауссовский шум для мультипликативной модели
    noise = np.random.normal(0, sigma, image.shape)
    noisy_image_float = image_float + image_float * noise

    # Обрезаем значения в [0, 1]
    noisy_image_clipped = np.clip(noisy_image_float, 0, 1.0)

    # Возвращаем к исходному диапазону и типу данных
    noisy_image = (noisy_image_clipped * 255).astype(image.dtype)
    return noisy_image

# 4. Другие типы шума (менее распространенные в общих задачах):
#    - Шум Пуассона (Poisson Noise / Shot Noise): Зависит от интенсивности сигнала, важен при слабом освещении (фотография, медицина).
#    - Квантования Шум (Quantization Noise): Возникает при преобразовании аналогового сигнала в цифровой с ограниченной битностью.
#    - Периодический Шум (Periodic Noise): Регулярные паттерны, часто из-за электрических помех. Удаляется с помощью Фурье-фильтрации.

# --------------------------------------------------

# Блок 3: Методы Фильтрации для Разных Типов Шума

# Выбор фильтра зависит от типа шума и необходимости сохранения деталей.

# 1. Для Гауссовского Шума:
#    - Усредняющий Фильтр (`cv2.blur`):
#      - Простое усреднение пикселей в окне.
#      - Эффективен для слабого шума, но сильно размывает края.
#      - `filtered = cv2.blur(noisy_image, (5, 5))`
#    - Гауссовский Фильтр (`cv2.GaussianBlur`):
#      - Взвешенное усреднение (веса по Гауссу).
#      - **Предпочтительный метод** для Гауссовского шума. Дает более гладкий результат и лучше сохраняет края, чем `cv2.blur`.
#      - `filtered = cv2.GaussianBlur(noisy_image, (5, 5), sigmaX=0)`
#    - Билатеральный Фильтр (`cv2.bilateralFilter`):
#      - Учитывает и пространственную близость, и схожесть интенсивностей.
#      - **Хорош, если нужно сохранить резкие края** при удалении Гауссовского шума.
#      - Медленнее, чем Гауссовский фильтр.
#      - `filtered = cv2.bilateralFilter(noisy_image, d=9, sigmaColor=75, sigmaSpace=75)`
#    - Фильтр Non-Local Means (NLM) (`cv2.fastNlMeansDenoisingColored` / `cv2.fastNlMeansDenoising`):
#      - Более продвинутый метод. Усредняет пиксели на основе схожести целых окрестностей (патчей), а не только отдельных пикселей.
#      - Может дать очень хорошие результаты для Гауссовского шума, хорошо сохраняя текстуры.
#      - Значительно медленнее предыдущих.
#      - `filtered = cv2.fastNlMeansDenoisingColored(noisy_image, None, h=10, hColor=10, templateWindowSize=7, searchWindowSize=21)` (параметры нужно подбирать)

# 2. Для Шума "Соль и Перец":
#    - Медианный Фильтр (`cv2.medianBlur`):
#      - **Наиболее эффективный и рекомендуемый метод**.
#      - Заменяет пиксель медианой соседей, эффективно игнорируя экстремальные значения (0 и 255).
#      - Хорошо сохраняет края.
#      - `filtered = cv2.medianBlur(noisy_image, 5)` (размер ядра должен быть нечетным)
#    - Линейные фильтры (Усредняющий, Гауссовский) **плохо** справляются с этим типом шума, так как они просто размазывают "соль" и "перец", создавая серые пятна.

# 3. Для Пятнистого Шума:
#    - Удаление пятнистого шума сложнее из-за его мультипликативной природы.
#    - Логарифмическое Преобразование + Линейный Фильтр:
#      - Иногда применяют логарифм к изображению, чтобы преобразовать мультипликативный шум в аддитивный.
#      - Затем применяют Гауссовский или другой фильтр к логарифмированному изображению.
#      - После фильтрации применяют экспоненциальное преобразование для возврата в исходное пространство.
#    - Специализированные Фильтры: Фильтр Ли (Lee filter), Фильтр Фроста (Frost filter), Фильтр Куана (Kuan filter) - часто используются в обработке SAR изображений. (Менее доступны в стандартном OpenCV).
#    - Адаптивные Фильтры: Фильтры, параметры которых меняются в зависимости от локальных характеристик изображения (например, дисперсии).
#    - Гауссовский или Медианный фильтр могут дать *некоторое* улучшение, но не являются оптимальными.

# --------------------------------------------------

# Блок 4: Библиотеки (Повторение)

# - **OpenCV (`cv2`)**: Основной инструмент для стандартных фильтров (`blur`, `GaussianBlur`, `medianBlur`, `bilateralFilter`, `fastNlMeansDenoising...`). Быстро и эффективно.
# - **SciPy (`scipy.ndimage`)**: Альтернатива для `gaussian_filter`, `median_filter`, `uniform_filter`.
# - **Scikit-image (`skimage.filters`, `skimage.restoration`)**: Предлагает Гауссовы, медианные фильтры, а также более продвинутые, как билатеральный, NLM (`denoise_nl_means`), BM3D (`denoise_wavelet` с опцией `denoise_pro` может использовать BM3D, если установлен), Total Variation Denoising (`denoise_tv_chambolle`).
# - **Kornia**: Для GPU-ускоренной фильтрации в пайплайнах PyTorch.

# --------------------------------------------------

# Блок 5: Пример Задачи и Решения (Сравнение Фильтров для Разных Шумов)

# --- Условие Задачи ---
# Задача: Загрузить изображение. Создать две копии: одну с Гауссовским шумом, другую с шумом "Соль и Перец".
# Применить Гауссовский и Медианный фильтры к *каждой* зашумленной копии.
# Визуально сравнить результаты, чтобы увидеть, какой фильтр лучше работает для какого типа шума.

# --- Решение (Полный Код) ---

import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# --- Функции добавления шума (из Блока 2) ---
def add_gaussian_noise(image, mean=0, sigma=25):
    if image.dtype != np.float64: image_float = image.astype(np.float64)
    else: image_float = image.copy()
    noise = np.random.normal(mean, sigma, image.shape)
    noisy_image_float = image_float + noise
    noisy_image_clipped = np.clip(noisy_image_float, 0, 255)
    noisy_image = noisy_image_clipped.astype(image.dtype)
    return noisy_image

def add_salt_pepper_noise(image, salt_prob=0.02, pepper_prob=0.02):
    noisy_image = np.copy(image)
    total_pixels = image.size
    num_salt = int(salt_prob * total_pixels)
    num_pepper = int(pepper_prob * total_pixels)
    # Соль
    salt_coords = [np.random.randint(0, i - 1, num_salt) for i in image.shape[:2]]
    if len(image.shape) == 3: noisy_image[salt_coords[0], salt_coords[1], :] = 255
    else: noisy_image[salt_coords[0], salt_coords[1]] = 255
    # Перец
    pepper_coords = [np.random.randint(0, i - 1, num_pepper) for i in image.shape[:2]]
    if len(image.shape) == 3: noisy_image[pepper_coords[0], pepper_coords[1], :] = 0
    else: noisy_image[pepper_coords[0], pepper_coords[1]] = 0
    return noisy_image

# --- 1. Загрузка Изображения ---
image_path = 'path/to/your/image.jpg' # ЗАМЕНИТЕ НА ВАШ ПУТЬ
if not os.path.exists(image_path):
    print(f"Ошибка: Файл не найден: {image_path}. Используется заглушка.")
    original_image = np.full((200, 300, 3), 128, dtype=np.uint8)
else:
    original_image = cv2.imread(image_path)
    if original_image is None:
        print(f"Ошибка: Не удалось загрузить: {image_path}")
        exit()
    print(f"Изображение '{image_path}' загружено.")

original_image_rgb = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

# --- 2. Добавление Разных Типов Шума ---
gaussian_noisy_image = add_gaussian_noise(original_image_rgb, sigma=30)
sp_noisy_image = add_salt_pepper_noise(original_image_rgb, salt_prob=0.03, pepper_prob=0.03)
print("Гауссовский шум и шум 'Соль и Перец' добавлены.")

# --- 3. Применение Фильтров к Обоим Типам Шума ---
# Фильтры для Гауссовского шума
gauss_filtered_g = cv2.GaussianBlur(gaussian_noisy_image, (5, 5), 0)
median_filtered_g = cv2.medianBlur(gaussian_noisy_image, 5)

# Фильтры для шума "Соль и Перец"
gauss_filtered_sp = cv2.GaussianBlur(sp_noisy_image, (5, 5), 0)
median_filtered_sp = cv2.medianBlur(sp_noisy_image, 5)
print("Фильтры применены к зашумленным изображениям.")

# --- 4. Отображение Результатов ---
plt.figure(figsize=(15, 10))

# Оригинал
plt.subplot(2, 3, 1)
plt.imshow(original_image_rgb)
plt.title('Original')
plt.axis('off')

# Результаты для Гауссовского шума
plt.subplot(2, 3, 2)
plt.imshow(gaussian_noisy_image)
plt.title('Gaussian Noise Added')
plt.axis('off')

plt.subplot(2, 3, 3)
plt.imshow(gauss_filtered_g)
plt.title('Gaussian Filter on Gaussian Noise')
plt.axis('off')

# Результаты для шума "Соль и Перец"
plt.subplot(2, 3, 5)
plt.imshow(sp_noisy_image)
plt.title('Salt & Pepper Noise Added')
plt.axis('off')

plt.subplot(2, 3, 6)
plt.imshow(median_filtered_sp)
plt.title('Median Filter on S&P Noise')
plt.axis('off')

# Покажем также "неправильный" фильтр для сравнения
plt.subplot(2, 3, 4) # Поместим сюда результат Median на Gaussian
plt.imshow(median_filtered_g)
plt.title('Median Filter on Gaussian Noise')
plt.axis('off')

# Можно добавить еще один ряд или заменить пустой график
# plt.subplot(2, 3, 4) # Пустое место или оригинал снова
# plt.axis('off')

plt.tight_layout()
plt.show()

print("Отображение результатов завершено.")
# Выводы из визуализации:
# - Гауссовский фильтр хорошо сглаживает Гауссовский шум (subplot 2,3,3).
# - Медианный фильтр отлично удаляет шум "Соль и Перец" (subplot 2,3,6).
# - Медианный фильтр не очень хорошо справляется с Гауссовским шумом (subplot 2,3,4) - он может немного сгладить, но не так эффективно, как Гауссовский.
# - Гауссовский фильтр плохо удаляет "Соль и Перец" - он размазывает точки шума (не показано явно, но можно добавить subplot для gauss_filtered_sp).

# --- Конец Примера ---

# --------------------------------------------------